### 参数设置

In [1]:
import os
import re
import sys
import time
import json
import jsonlines
from tqdm import tqdm
import copy
#from openai import AzureOpenAI
# from langchain_openai import AzureChatOpenAI
from datetime import datetime

from langchain.prompts import PromptTemplate 
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

parent_dir = os.path.dirname(os.path.abspath(''))
sys.path.append(parent_dir)
from llm import AzureLLM, GPTProxyLLM, DeepSeekLLM

### 键值对模板、超参数、抽取列表

In [19]:
# 可配置选项


data_folder = "../extra_data"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# 合并粒度(推荐1-4，越小划分的越细,每n章合并,这个参数为0时,代表全文合并；参数为1时，代表保持原主线结构不变)
# 这个参数为0时,代表全文合并,这个参数需要小于seg_main_plot_num

combine_plot_num = 1
combine_plot_num = 0

# 重要分支 - 剧情长度
# 小说划分粒度,主线划分为多少章节(推荐6-10)
seg_main_plot_num = 10
#seg_main_plot_num = 6

# 重要参数 - 语言
# 语言(0时中文，1是english，只接受这两个参数)

use_language = 0
#use_language = 1

# 多线剧情发散度（生成结局数量，等？）(生成范围/+1)
branch_num = 2
#role_num = 3


# 重要参数 - 剧情分枝
# 生成多线还是单线剧情
# true是单线

# if_single_plot = True
if_single_plot = False



if_role_known = True

# 是否使用小说文本
#if_use_novel_doc = False
#if_use_novel_doc = True
# 小说中是否具有额外信息还是需要全部自己抽取
#if_novel_extra_info = False


gen_parameter_dict = {"seg_main_episode_num":seg_main_plot_num,"combine_episode_num":combine_plot_num,"branch_num":branch_num,"language":use_language,"if_single_episode":if_single_plot}
llm_model_name = "deepseek_r1"

if use_language == 0:
    language = "中文"
elif use_language == 1:
    language = "English"
else:
    print(f"warning! wrong parameter for language {use_language}")
if combine_plot_num>seg_main_plot_num or combine_plot_num<0:
    print(f"warning! wrong parameter for combine_plot_num {combine_plot_num}")


# 键值对模板
# story，plot，messages
template_dict_story = {
    "Settings":{
        "story_id": "",         # 唯一id，用于查找
        "story_name": "",       # 故事名
        "story_style": "",      # 故事风格，尽可能超过5个短语
        "leading_role":"",      # 每个episode中都要出现主角
        "story_desc": "",       # 故事的主要描述
        "story_goal":"",      # 故事主角需要做的目标
        "story_chars": {},      # dict, 故事有哪些主角,性格以及人物描述 
        "storylines":[],      # list, 故事主线梗概
        "states":[],            # 由多个list组成, 每个list代表这一个剧情顺序
        "story_type":"",        # 故事类型，从哪里获得的，game/novel...
        "story_struct":"",         # 故事结构，单线/多线
        "story_language":"",          # 故事语言，中文/英文
        "gen_parameter":{}      # 生成故事使用的超参数
    },
    "Episodes":[]   # plots节点, plot 是list，多个下面的plot
}
template_dict_plot = {
    "episode_id": "",
    "pre_episode_id":[],
    "episode_goal": "",
    "episode_scene": "",
    "episode_desc": "",
    "episode_chars": {},
    "extra_info": [],
    "triggers": [
#        {
#            "condition": "",
#            "examples": [],
#            "next_episode": "",
#        },
    ],
    "message_pairs":[],
}
template_dict_message =  {
    "task": ["transition", "role-play", ""],
    "user": "@npc content_text",
    "bot": {
        "narrative": "", 
        "role_dialogue": [
            {"name": "", 
             "utterance": "",
             },
        ],
        "next_episode": ""
    }                       
}


# 游戏列表,推荐剧情为主的rpg(Role-playing game),act(Action Game),avg(Adventure Game)

# 
# rts 会有操作方面的提示，建议删除
game_name_list = [

    #"星际争霸2：自由之翼(StarCraft 2: Wings of Liberty)",
    #"星际争霸2：虫群之心(StarCraft 2: Heart of the Swarm)",
    #"星际争霸2：虚空之遗(StarCraft 2: Legacy of the Void)",
    #"魔兽争霸3：混乱之治(Warcraft 3: Reign of Chaos)",
    #"魔兽争霸3：冰封王座(Warcraft 3: The Frozen Throne)",
     
    "生化危机7(Resident Evil 7)", 
    #"生化危机8(Resident Evil 8)", 
    #"生化危机2重制版(Resident Evil 2 Remake)", 
    #"生化危机4重制版(Resident Evil 4 Remake)", 
     
    #"古墓丽影9(Tomb Raider)",
    #"古墓丽影：崛起(Rise of The Tomb Raider)",
    #"古墓丽影：暗影(Shadow of The Tomb Raider)",

    "死亡搁浅(Death Stranding)",
    "双人成行(It Takes Two)",
    "行尸走肉(The Walking Dead)", 
    "夜之城:2077(Cyberpunk 2077)",
     
    "巫师三:狂猎(The Witcher 3: Wild Hunt)", 
    "巫师二:刺客之王(The Witcher 2: Assassins of Kings)", 

    "战神：诸神黄昏(God of War: Ragnarok)",
    "鬼泣5(Devil May Cry 5)",
    "尼尔：机械纪元(NieR: Automata)",
     
    "荒野大镖客2(Red Dead Redemption 2)",
    
    "逃生(Outlast)",
    "逃生2(Outlast 2)",
    "恶灵附身(The Evil Within)",
    "控制(Control)",

    "消逝的光芒1(Dying Light)",
    "消逝的光芒2:人与仁之战(DyingLight2:StayHuman)",
    "最后生还者(The Last of Us)",
    "最后生还者2(The Last of Us Part 2)",
    "看门狗2(Watch_Dogs 2)",
    "最后生还者2(The Last of Us Part 2)",
    "只狼：影逝二度(Sekiro：Shadows Die Twice)",
     
    "龙腾世纪:审判(Dragon Age: Inquisition)",  
    "质量效应(Mass Effect)",  # 有一系列
    "异界锁链(Chains of Promathia)", 
    "最终幻想 XV(Final Fantasy XV)", 
    "辐射：新维加斯(Fallout: New Vegas)",  
]
# 错误案例，这种类型建议删除
# "仙剑奇侠传1(Chinese Paladin: Sword and Fairy1)"  # 中文游戏gpt不了解，以及剑侠情缘等国产游戏
# "底特律：变人(Detroit: Become Human)",            # 这个游戏是多线叙事, 从三个主角视角出发

In [4]:
# if llm_model_name == "deepseek_r1":
#     llm_model = DeepSeekLLM()
#     print("llm:", llm_model)
# else:
#     print("llm model name error")

llm_gpt = GPTProxyLLM()
llm_deepseek = DeepSeekLLM()

In [5]:
llm_deepseek.chat("1")

'Alright, the user just sent "1". That\'s pretty vague. I need to figure out what they\'re asking for.\n\nMaybe they\'re testing if the AI is working? Or perhaps they meant to type something else but hit send too soon.\n\nI should ask for clarification to make sure I understand their request. Let me respond in Chinese since the user used Chinese in their message earlier.\n<thinking>\n您好！看起来您输入了数字“1”。请问有什么具体的问题或需要帮助的地方吗？我会尽力为您解答！'

### 基础剧情线生成
小说抽取plot后也从这里开始

In [4]:
# 抽取剧情相关信息
prompt_background=PromptTemplate(
    template = """请依次回答介绍在{novel_name}游戏中以下内容：
游戏的故事风格(使用几个短语形容，使用逗号隔开,7个词以上)，发生的背景(时间地点人物, 以第三人称风格介绍)，游戏剧情中主角的主要目标，主角分别是什么
介绍时不要出现“玩家”，“游戏”等令人出戏的词语
保持结果的格式和例子中的格式一致, 并以键值对的json格式: '{{key:value, ...}} '重构。
用{language}回答,不要提到玩家等令人出戏的字眼
下面是一个例子（取自游戏"巫师3"）
{{
    "story_style":"奇幻,中世纪，成人向叙事,复杂阴谋,黑暗幻想,史诗冒险,传奇主义色彩,哥特式神秘感,魔法与剑",
    "background":"这是一个辽阔的幻想世界，这个世界充满了魔法、怪兽和战争，猎魔人杰洛特是这个世界中的一名传奇人物，他的养女希里拥有强大的魔法能力，被来自另一个维度，拥有强大的力量且神秘的狂猎势力追捕。杰洛特为了寻找并保护希里，踏上了一段跨越战争蹂躏的大陆的冒险旅程，同时卷入了北方王国与尼尔夫加德帝国之间的冲突。猎魔人是被训练来对抗怪物的变种人类，尽管他们拥有超自然力量，却常被社会边缘化。",
    "target":"杰洛特的主要目标是是寻找并保护他的养女希里，她是一个拥有强大力量的源术士，目前正在被神秘的狂猎骑士团追捕，同时杰洛特还需要揭露狂猎的计划，最终阻止他们。在这个过程中需要探索一个充满危险和奇幻元素的广阔开放世界，解决各种任务和委托，与怪物战斗，同时做出影响剧情走向的决策。杰洛特还需要不断提升自己的能力，包括剑术、魔法和装备，以应对越来越强大的挑战。",
    "leading_role":"杰洛特"
}}
    """,
    input_variables=["novel_name","language"],
)
# 请以主角的视角第三人称进行叙述
# 《巫师3：狂猎》设定在一个辽阔的幻想世界，游戏的背景承接自《猎魔人》系列小说和前两部游戏，



# 抽取剧情主要剧情线
prompt_main_plot=PromptTemplate(
    template="""游戏{novel_name}的主要剧情线是什么? 请完整介绍最主要的剧情线（单线）,请尊重原始剧情，不要遗漏重要情节
需要每章节内容均衡，所有章节总体可以概括整个剧情，维持总结后的章节数在{seg_main_plot_num}章左右
在各个plot的情节中应该有所差异，但是临近的plot应该有所关联
注意,在每个plot中都需要出现story中最重要的主角{story_leading_role},需要强调主角

使用如下格式`总结:详细内容`，保持结果的格式和例子中的格式一致, 并整体以json格式: '{{"main":[xx:xx, xx:xx, ...]}} '重构。 
使用{language},下面是一个例子：
寻找希里:主角杰洛特寻找他的养女希里，发现她被狂猎追捕。
追踪狂猎:杰洛特开始调查狂猎的活动，了解他们的目标和动机。
政治纷争:杰洛特在调查狂猎时卷入北方王国之间的战争与政治斗争，选择支持的势力。
...
    """,
    #input_variables=["novel_name","language"],
    input_variables=["novel_name","seg_main_plot_num","story_leading_role","language"],
)    

# 维持总结后的章节数在{seg_main_plot_num}章左右
# 不能低于{seg_main_plot_num}个plot，根据剧情长度适当增加plot数量



# # 剧情重组织之后生成，目前未使用
# prompt_combine_plot=PromptTemplate(
#     template = """进一步的，在{novel_name}游戏中，上述答案中的 {plot_combine}部分，更加详细的剧情是什么? 重新划分段落详细叙述，每行所占的剧情数应该相同
#         每个剧情前用(plot *),*是对应序号，表示这是第几幕出现的，序号应该连续递增，从1开始
#         剧情大概有{seg_main_plot_num}个
            
#         仅仅解释{plot_combine}的部分，不要与其他部分内容重合
#         使用如下格式`总结:详细内容`, 保持结果的格式和例子中的格式一致, 并以json格式: 'main:[xx:xx, xx:xx, ...] '重构。 
#         用{language}回答,下面是一个例子:

#         (plot 1)调查遗迹:探索古代遗迹，了解希里与古老力量的关系。
#         (plot 2)遭遇敌人:面对追寻希里的敌对势力，保护自己和寻找线索。 
#         (plot 3)发现藏身处:最终找到希里的一些藏身之处，获取更深入的信息。
#         (plot 4)...
#         """,
#     input_variables=["novel_name","plot_combine","seg_main_plot_num","language"],
# )



In [5]:
# 这里应该有个大循环，以便程序一键执行所有的游戏列表
pass

# 生成内容


In [22]:

# 拆分think
def ds_think_analysis(model_chat:str):
    
    model_chat_list = model_chat.split("<thinking>") 
    if len(model_chat_list) == 2:
        model_think = model_chat_list[0].strip()
        model_ans = model_chat_list[1].strip()
    elif len(model_chat_list) == 1:
        model_think = ""
        model_ans = model_chat_list[0].strip()
        # print("Warning! no <thinking> in res")
    else:
        print("Warning! too many <thinking> in res")
        
    return model_think, model_ans




# llm及处理
def llm_chat_analysis(llm_model_name:str, llm_prompt:str, res_json_type:str):
    # print(llm_prompt)
    if llm_model_name == "deepseek-r1":
        llm_model = DeepSeekLLM()
    elif llm_model_name == "gpt-4o":
        llm_model = GPTProxyLLM()
    else:
        print("wrong llm model type")

    res_llm = llm_model.chat(llm_prompt)
    # print(res_llm)

    if llm_model_name == "deepseek-r1":
        res_think, res_ans = ds_think_analysis(res_llm)
        res_think = res_think.replace('\n\n','\n').strip()
    else:
        res_think = ""
        res_ans = res_llm
    res_ans = res_ans.strip()

    try:
        cleaned_json_string = res_ans.split('```json')[1].split('```')[0].strip()
    except:
        try:
            cleaned_json_string = res_ans.split('```')[1].strip()
        except:
            cleaned_json_string = res_ans
    cleaned_json_string = cleaned_json_string.strip()

    if res_json_type == "str":
        return res_think, cleaned_json_string
    if res_json_type == "int":
        return res_think, int(cleaned_json_string)
    # print(cleaned_json_string)
            
    try:
        res_json_load = json.loads(cleaned_json_string)
    except:
        try:
            cleaned_json_string1 = cleaned_json_string
            if not cleaned_json_string1.startswith("{"):
                cleaned_json_string1 = "{\n" +cleaned_json_string1
            if not cleaned_json_string1.endswith("}"):
                cleaned_json_string1 = cleaned_json_string1 + "\n}" 
            res_json_load = json.loads(cleaned_json_string1)
        except:
            try:
                cleaned_json_string1 = cleaned_json_string
                cleaned_json_string1 = cleaned_json_string1.strip("\"").strip("main").strip("\"").strip(":")
                res_json_load = json.loads(cleaned_json_string1)
            except Exception as e:
                print(type(cleaned_json_string))
                print(f"wrong res type: \n{cleaned_json_string}")
                print("发生错误：", e)
                return res_think, cleaned_json_string
        
    if type(res_json_load) == dict:
        try:
            res_json_load = res_json_load["main"] 
        except:
            pass
    
    
    if res_json_type == "dict":
        res_json = {}
        if type(res_json_load) == dict:
            res_json = res_json_load
        elif type(res_json_load) == list:
            for item in res_json_load:
                # 多个str组成的list：['伊森·温特斯: 主角...',...]
                if type(item) == str:
                    item1 = item.split(":")
                    res_json[item1[0].strip()] = item1[1].strip()
                # 多个dict组成的list：[{'伊森·温特斯': '主角...'},...,{}]
                elif type(item) == dict:
                    res_json.update(item)
        # 几率低, 比较粗糙的处理方法, 可以改进
        # elif type(res_json_load) == str:
        #     pass
        else:
            print(f"Warning! res_json_load type error {type(res_json_load)}, need {res_json_type}")

    elif res_json_type == "list":
        res_json = []
        if type(res_json_load) == list:
            res_json = res_json_load
        elif type(res_json_load) == dict:
            res_json = []
            for key in res_json_load.keys():
                res_json.append(f"{key}: {res_json_load[key]}")
        # 几率低, 比较粗糙的处理方法, 可以改进
        elif type(res_json_load) == str:
            res_json.append(res_json_load)
        else:
            print(f"Warning! res_json_load type error {type(res_json_load)}, need {res_json_type}")

    return res_think, res_json

In [7]:
# 背景 等内容dict填充

novel_name = game_name_list[0]
res_dic = {}
print(f"游戏名称：{novel_name}")

time_background_begin = time.time()

# chain0 = prompt_background | llm_agent
# res0_ans = chain0.invoke({"novel_name":novel_name, "language":language}).content
# try:
#     cleaned_json_string0 = res0_ans.split('```json')[1].split('```')[0].strip()
# except:
#     cleaned_json_string0 = res0_ans

res0_dict = {}
prompt0 = prompt_background.format(novel_name=novel_name, language=language)
res_think, res0_dict = llm_chat_analysis("gpt-4o", prompt0,"dict")


story_dict = copy.deepcopy(template_dict_story)

story_dict["Settings"]["story_name"] = novel_name
style_items = re.split(r'[，,]', res0_dict["story_style"])
story_dict["Settings"]['story_style'] = [item.strip() for item in style_items if item.strip()]

# 故事介绍是是背景还是背景加目标？
story_dict["Settings"]['story_desc'] = res0_dict["background"]
story_dict["Settings"]["story_goal"] = res0_dict["target"]
#story_dict["story_info"]['story_desc'] = res0_dict["background"] + '/n' + res0_dict["target"]

story_dict["Settings"]["leading_role"] = res0_dict["leading_role"]
story_leading_role = res0_dict["leading_role"]
story_dict["Settings"]["story_type"] = "game"
story_dict["Settings"]["story_struct"] = "single" if if_single_plot else "multi"
story_dict["Settings"]["story_language"] = language
story_dict["Settings"]["gen_parameter"] = gen_parameter_dict

time_background = time.time() - time_background_begin
print(f"背景信息抽取时间：{time_background:.2f}秒")
print(json.dumps(story_dict,indent=2, ensure_ascii=False))

# 背景生成ds耗时 60s - 100s
# 背景生成gpt耗时 5s - 10s

游戏名称：生化危机7(Resident Evil 7)
背景信息抽取时间：5.00秒
{
  "Settings": {
    "story_id": "",
    "story_name": "生化危机7(Resident Evil 7)",
    "story_style": [
      "恐怖生存",
      "心理惊悚",
      "第一人称视角",
      "紧张氛围",
      "超自然元素",
      "家庭悲剧",
      "探索与解谜"
    ],
    "leading_role": "伊森·温特斯",
    "story_desc": "故事发生在2017年，美国路易斯安那州的一个偏远农场。伊森·温特斯是一名普通的男子，他收到了一段来自失踪三年的妻子米娅的神秘录像，录像中米娅请求他来找她。伊森追踪线索来到贝克家族的庄园，这个地方被一种神秘的生物感染，贝克一家也因此变得疯狂而暴力。伊森在庄园中遭遇了各种恐怖事件，并逐渐揭开隐藏在背后的秘密。",
    "story_goal": "伊森的主要目标是找到并拯救他的妻子米娅，同时揭开贝克家族庄园中恐怖事件背后的真相。在这个过程中，他需要面对被感染的贝克家族成员和其他怪物，寻找线索和资源以求生存，并最终逃离这个恐怖的地方。",
    "story_chars": {},
    "storylines": [],
    "states": [],
    "story_type": "game",
    "story_struct": "multi",
    "story_language": "中文",
    "gen_parameter": {
      "seg_main_episode_num": 10,
      "combine_episode_num": 0,
      "branch_num": 2,
      "language": 0,
      "if_single_episode": false
    }
  },
  "Episodes": []
}


In [8]:
# 游戏主线

# chain1 = prompt_main_plot | llm_agent
# res1 = chain1.invoke({"novel_name":novel_name, "seg_main_plot_num":seg_main_plot_num, "story_leading_role":story_leading_role,"language":language}).content
# #print(res1)
# try:
#     cleaned_json_string1 = res1.split('```json')[1].split('```')[0].strip()
# except:
#     cleaned_json_string1 = res1
#print(cleaned_json_string1)

time_mainplot_begin = time.time()
prompt1 = prompt_main_plot.format(novel_name=novel_name, seg_main_plot_num=seg_main_plot_num, story_leading_role=story_leading_role, language=language)
res_think, main_plot_list_re = llm_chat_analysis("gpt-4o", prompt1, "list")

time_mainplot = time.time() - time_mainplot_begin
print(f"主线剧情抽取时间：{time_mainplot:.2f}秒")

print("主线剧情: ",json.dumps(main_plot_list_re,indent=2, ensure_ascii=False))
story_dict["Settings"]["storylines"] = main_plot_list_re

# 主线剧情生成ds耗时 60s - 100s
# 主线剧情生成gpt耗时 6s - 10s

主线剧情抽取时间：5.67秒
主线剧情:  [
  "收到求救信:伊森·温特斯收到失踪三年的妻子米娅的求救信，前往路易斯安那州的贝克家族庄园寻找她。",
  "进入庄园:伊森进入贝克家族庄园，发现庄园荒废且充满恐怖氛围，并找到被囚禁的米娅。",
  "米娅的变化:伊森试图带米娅逃离，但米娅突然变得狂暴并攻击伊森，伊森被迫自卫。",
  "贝克家族:伊森被贝克家族抓住，见到了家族成员杰克、玛格丽特和卢卡斯，他们都表现出非人类的特性。",
  "逃离杰克:伊森设法从杰克的追捕中逃脱，并在庄园中寻找线索，揭示贝克家族的秘密。",
  "玛格丽特的威胁:伊森深入庄园，面对玛格丽特的攻击，并发现她与一种神秘生物有关。",
  "卢卡斯的陷阱:伊森解开卢卡斯设置的致命谜题，逐步揭露贝克家族与生化武器的联系。",
  "伊芙琳的真相:伊森发现贝克家族受控于一种名为伊芙琳的生化武器，她是导致一切的根源。",
  "对抗伊芙琳:伊森与伊芙琳展开最终对抗，利用找到的血清阻止她的计划。",
  "重聚与逃离:伊森成功击败伊芙琳，救出米娅，并在政府特工的帮助下逃离庄园，重获自由。"
]


In [9]:
# - 剧情重写：对当前剧情树添加新剧情，产生额外分支剧情，新结局，并使所有剧情连贯，使其包含多个分支（这里的分支不只是结局，包括中间也可能产生分支）。
# - 详细描述：详细写下每个情节和结局，包括发生的事件、跳转原因、触发条件。需要考虑不同选择导致的剧情走向的改变。
# - 角色一致：确保新情节与角色特点相符。如果两个分支指向同一剧情，它们的跳转逻辑必须各自合理, 且所有的剧情都应该围绕主角{leading_role}展开
# - 剧情连贯：确保从起始事件plot1可以达到其他所有事件，而且所有事件都可以通过不同的决策路径到达最终几个结局，plot的数量应该比end多。
# = 每个plot必须可以跳转到其他事件，也就是from plot *，*可以覆盖所有plot节点（不要在from后写跳转原因）
# - 结局多样：创造显著不同的多个结局，结局不能同质化，确保不同结局的人物命运和事件结果有显著差异（但应该符合剧情人设以及背景）。确保不同的重要事件能触发不同的事件链，导致多样的结局
# - 新的剧情是树状结构，不要产生环，也就是除了第一个节点外每个plot的from应该只有1个
# - 结局数量控制：结尾的数量不能超过4个，且每个分支长度不能短于4个章节，每某一事件产生的分支剧情的数量不能超过2个，总情节数不要超过15个
# - 不同plot之间的内容不能重复或者近似
# - 在生成新剧情时，必须保留原有剧情的关键节点和结局，不能完全重写或忽略原有剧情。新剧情应在原有剧情的基础上进行扩展和分支。需要在原有剧情之间添加新剧情，可以改变原有剧情序号（但是需要保留原先剧情以及结局）       
# - 生成的新剧情不能与原有剧情重复，尤其是前几个plot不能直接复制原有剧情的内容。新剧情应从原有剧情的某个节点开始分支，而不是从头开始重复。
# - 每个分支的跳转逻辑必须清晰合理，确保玩家能够理解不同选择导致的剧情走向变化。

### 多线剧情重组织

1. 将单线剧情重新组合，重写多线剧情
2. 将多线剧情线通过dict进行连接

In [10]:
# 重写多线剧情prompt

# 实现有些困难
# 跳转合理：如果两个分支指向同一个剧情，那么这两个分支的跳转必须符合不同分支的剧情逻辑。所有结局要和对应剧情逻辑保持一致。
prompt_gen_multi_plot=PromptTemplate(
    template = """
你是一个剧情策划，需要将我给你的单线剧情重构为多分支叙事结构，单线剧情是{novel_name}游戏中的一部分

# 核心原则
1. 剧情重写
 - 新剧情需要在原有剧情的基础上进行扩展和分支，产生额外分支剧情和新结局
 - 详细描述每个情节和结局，包括发生的事件、跳转原因、触发条件
 - 所有的剧情都应该围绕主角{leading_role}展开,且主角应该向游戏目标{game_target}努力
 - 你的输出包含两种节点: plot节点和end节点。plot节点是中间剧情节点; end是结局节点, 作为一条故事线的结束
2. 保持原始骨架
 - 必须保留原有剧情和结局，不能完全重写或忽略原剧情
 - 新分支应从原剧情的中间节点开始产生，而不是只在原剧情结束后才产生
 - 原最终节点必须作为新分支的第一个结局(end 1)
3. 三维叙事设计
 - 横向拓展: 每个节点不超过2个分支, 不同plot之间的内容不能重复或者近似
 - 纵向延伸: 每个分支需包含4-6个递进plot节点后到达结局, 在原有前期剧情(前3个章节)中间, 应该至少出现一个新分支plot, 即第一个你创造的节点分支在plot2-3里面
 - 垂直深化: 确保每个结局体现: 人物命运的根本性差异(生存/死亡/蜕变)/世界观的不同侧面展现, 结局不能同质化，确保不同结局的人物命运和事件结果有显著差异; 在分支节点有核心冲突的差异化解决方案
4. 因果链约束
 - 每个选择必须触发【蝴蝶效应】，使后续剧情产生本质区别
 - 当一个plot面临抉择时, 其必须产生不同节点的分支
 - 产生的选择必须满足: 前序决策逻辑不冲突; 角色行为模式保持一致; 世界观设定无矛盾; 剧情保持连贯
 - 每个分支的跳转逻辑必须清晰合理，确保玩家能够理解不同选择导致的剧情走向变化。

# 技术规范
1. 结构要求
 - 节点数量: plot不要超过18个, end不能超过3个(即剧情线不要超过3条)
 - 路径唯一: 不能产生环结构, 除起始节点(没有from)外, 其他每个节点有且只能有一个前置节点(from只有一个)；结局的数量就代表一共有几条剧情线
 - 树状结构: 确保从起始事件plot1(根节点)可以达到其他所有事件, 而且所有plot都可以通过路径到达至少一个end节点(plot可以看作是非叶子节点, end是叶子节点); (即没有前置节点或后续节点的plot)只有end才能作为最终结局
 - 禁止出现孤立plot: 对于除了plot1的plot(非叶子节点), 必须被一个其他节点指向(有且只能有1个前置节点); 必须至少有一个后续节点(至少有一个后续节点, 也就是个plot必须可以跳转到其他事件)
2. 内容规范
 - 每个分支plot需明确: 核心冲突(人物 vs 环境/他人/自我); 关键决策点(必须影响后续分支走向); 状态转变（角色能力/关系/认知的变化）
 - 每个end需体现: 核心矛盾的终结方式; 主角{leading_role}的最终状态; 对游戏目标{game_target}的达成程度

# 输出格式
严格遵循以下结构：
(plot *)【章节标题】: 核心事件描述。关键决策点(from plot *)
(end *)【结局名称】: 结局描述及对{game_target}的影响程度(from plot *)
具体要求: 
 - 在剧情后仅用(from plot **)指示从何plot跳转而来, 不要在from后写跳转原因
 - 所有行的开头都应该是(plot *)或者是(end *)，代表剧情节点和结局节点
 - 除了plot节点和end节点不要输出其他内容(比如设计原因, 总结等)
 - 你的输出应该从plot1开始, 序号需要连贯; 先输出完所有plot, 再输出end

示例: 
```
(plot 1)寻找希里:杰洛特寻找他的养女希里，她被狂猎追捕。(begin)
(plot 2)战场的鲜血:希里情绪低落，需要做出选择以将决定希里是否能够控制自己的力量。(from plot 1)
(plot 3)希里女王支线:"战场上的鲜血"任务选择去带希里参见皇帝。(from plot 2)
(plot 4)希里猎魔人支线:"战场上的鲜血"任务选择去追杀狂猎。(from plot 2)
(plot 5)为她铸剑:杰洛特为希里铸造一把钢剑，并在百果园的酒馆将剑送给她，象征着她成为一名真正的猎魔人。(from plot 4)
(plot 6)各奔东西:希里会独立去探索世界，而杰洛特则会继续他的猎魔人生涯。(from plot 5)
(plot 7)共同作战:希里与杰洛特一起成为猎魔人，为追杀狂猎做准备(from plot 5)
(plot 8)寻访猎魔大师:杰洛特听说陶森特有一位猎魔大师结束了混乱的吸血鬼屠杀事件，决定去找他交流经验(from plot 6)
(end 1)希里的堕落:希里在成为女王后，被权力所诱惑，与狂猎达成交易，最终同流合污(from plot 3)
(end 2)胜利与牺牲:虽然最终击败了狂猎，但是希里为了保护她所爱的人选择了牺牲自己(from plot 7)
(end 3)重见故友:杰洛特发现猎魔大师就是希里。她在庄园的树下与杰洛特重逢(from plot 8)
```
参考内容：
游戏发生的背景如下:{game_background}
用{language}回答,原单线剧情树如下:{main_plot_str}
    """,
    input_variables=["novel_name","main_plot_str","game_background","game_target","language","leading_role"],
 )  

# 在产生分支剧情时原剧情应该有所提示，分支剧情的输出位置尽可能贴近分支节点
# 比如在之前的剧情中间插入分支，而不是只在末尾加入分支，并将目前整体的剧情重构

In [11]:
# 希望直接生成中间剧情？
# plot需要重新组织
# 将单线剧情重新组合，重写多线剧情

#print("原单线剧情摘要: ")
#print(json.dumps(main_plot_list["main"],indent=2, ensure_ascii=False))


# 这里要注释!!! 

# if_single_plot = False

if not if_single_plot:
    #print("A")
    # 重写多分支剧情树
    time_gen_multi_begin = time.time()
    main_plot_str = ""
    main_plot_str = "\n".join(item for item in main_plot_list_re)
    #for index, item in enumerate(main_plot_list["main"], start=1):
    #    main_plot_str += f"{index}. {item}\n"
    #print(main_plot_str)

    # chain_gen_multi = prompt_gen_multi_plot | llm_agent
    # res_gen_multi = chain_gen_multi.invoke({"novel_name": novel_name, "main_plot_str": main_plot_str,"branch_num":branch_num, "branch_num_plus":branch_num+1,"game_background":story_dict["Settings"]["story_desc"],"game_target":story_dict["Settings"]["story_goal"],"language":language,"leading_role":res0_dict["leading_role"]})
    
    prompt2 = prompt_gen_multi_plot.format(novel_name=novel_name, main_plot_str=main_plot_str, game_background=story_dict["Settings"]["story_desc"], game_target=story_dict["Settings"]["story_goal"], language=language, leading_role=res0_dict["leading_role"])
    res_think, res_gen_multi = llm_chat_analysis("deepseek-r1", prompt2, "str")
    # res_think, res_gen_multi = llm_chat_analysis("gpt-4o", prompt2, "str")
    
    time_gen_multi = time.time() - time_gen_multi_begin
    print(f"多线剧情生成时间：{time_gen_multi:.2f}秒")
    print(res_think)
    print("重写后多线剧情:")
    print(res_gen_multi)

    # 多线剧情生成ds耗时 120s - 180s
    # 多线剧情生成gpt耗时 10s - 20s

多线剧情生成时间：64.60秒
好的，我现在需要处理用户的任务，将生化危机7的单线剧情重构为多分支叙事结构。首先，我得仔细阅读用户提供的核心原则和技术规范，确保完全理解要求。
首先，用户希望保留原始剧情骨架，这意味着必须保留原有的主要事件和结局，同时从中间节点开始扩展分支。原最终节点作为第一个结局，也就是end 1。然后，我需要从原剧情的中间部分，比如plot2到plot3之间，引入新的分支，确保每个分支有4-6个节点，最终导向不同的结局。
接下来，我需要考虑如何横向扩展，每个节点不超过两个分支，避免重复。同时，纵向延伸要求每个分支有足够的深度，每个选择触发蝴蝶效应，导致截然不同的后续发展。垂直深化方面，每个结局需要展示人物命运的根本差异和世界观的不同侧面，比如生存、死亡或蜕变。
然后，分析原剧情的关键节点。原剧情从收到求救信开始，进入庄园，米娅的变化，贝克家族出现，逃离杰克，玛格丽特的威胁，卢卡斯的陷阱，揭示伊芙琳的真相，对抗伊芙琳，最后逃离。原结局是伊森救出米娅并逃离，作为end 1。
现在需要在这些中间节点插入分支。比如，在plot3米娅攻击伊森时，是否选择信任她，可能导致不同的分支。或者在plot5逃离杰克时，选择不同的逃脱路线，可能影响后续遇到的角色和线索。
还要确保每个分支的核心冲突不同，比如人物vs他人，环境，或自我。例如，在某个节点，伊森可能面临是否使用血清的抉择，这会影响他后续的能力或结局。关键决策点必须合理，符合角色行为，并且导致后续剧情的重大变化。
然后，考虑技术规范，结构必须是树状，没有环，每个节点只有一个前置，结局不超过三个。需要确保所有plot节点都能导向至少一个end，并且从plot1开始连贯。
可能的分支点：在plot3米娅变化时，伊森可以选择是否杀死米娅，或者尝试拯救她，这会导致不同的分支。或者在对抗杰克时，选择不同的逃生路线，可能影响后续遇到的敌人和资源。
然后，设计每个分支的4-6个节点，比如选择信任米娅，可能导致她成为盟友，但后续发现她被控制，需要新的冲突。或者选择不信任，导致独自行动，但资源不足，面临更多危险。
同时，结局需要差异明显，比如end1是原结局，end2可能是伊森被感染成为怪物，end3可能是米娅死亡，伊森独自逃离但真相未明。
需要检查每个选择是否符合因果链，不会出现逻辑矛盾，并且角色行为一致。例如，如果伊森在某

In [12]:
# 将多线剧情树结构反向组织
# 从form plot 变成指向下一个的dict
# 多线情况下，反向索引

def filter_plot_end(ans:str):
    new_ans = ""
    temp_list = ans.split('\n')
    for temp in temp_list:
        temp = temp.replace('（','(').replace('）',')').replace("：",":").strip()
        if temp:
            if temp.startswith('(') and temp.endswith(')'):
                new_ans += temp + '\n'
    return new_ans.strip()

def extra_plot_end(item_str:str, plot_sum_last:int):
    item_str = item_str.replace("(P","(p").replace("(E","(e").replace("(F","(f").strip()
    
    assert item_str != "" and item_str[0]=="("
    
    from_plot = item_str.split("from plot ")[1].split(")")[0] if "from plot " in item_str else 0        # 提取来源节点编号
    episode_key_abstract = item_str.split(")",1)[1].rsplit("(")[0].strip()          # 提取实际内容
    from_plot_list = []
    # from 多个plot的情况
    try:
        from_plot_list = [int(from_plot)] if from_plot else [0]
    except:
        num_temp = re.findall(r'\d+', from_plot)
        from_plot_list = [int(item) for item in num_temp]
        try:
            from_plot_list = [int(from_plot)] if from_plot else [0]
        except:
            num_temp = re.findall(r'\d+', from_plot)
            from_plot_list = [int(item) for item in num_temp]
        
    if item_str.startswith("(plot"):
        plot_num = int(item_str.split("plot ")[1].split(")")[0])        # 提取剧情编号
        # 判断一下，最大的plot是多少,这一步是判断一下gpt输出
        if plot_num > plot_sum_last:
            plot_sum_last = plot_num
        elif plot_num == plot_sum_last:
            print(f"注意: 出现相同plot序号 {plot_num}")
        plot_dict = {
            "node_type":"plot",
            'plot_num': plot_num,
            'from_plot': from_plot_list,
            'episode_key_abstract': episode_key_abstract
        }
    elif item_str.startswith("(end"):
        plot_num = int(item_str.split("end ")[1].split(")")[0])      # 提取结局剧情编号
        plot_dict = {
            "node_type":"end",
            'end_num': plot_num,
            'from_plot': from_plot_list,
            'episode_key_abstract': episode_key_abstract
        }
    else:
        print(f"no right type, the string is: \"{item_str}\"")
        plot_dict = {}
        plot_num = -1
    return plot_dict,plot_num,plot_sum_last




In [ ]:
# 将多线剧情树结构反向组织
# 从form plot 变成指向下一个的dict
# 多线情况下，反向索引

# if_single_plot = True
# if_single_plot = False
if not if_single_plot:
# if False:

    pre_plot_dict = {}
    pre_end_dict = {}

    # new_gen_multi_plot = res_gen_multi.content
    #print(new_gen_multi_plot)
    #new_gen_multi_plot = new_gen_multi_plot.split('```json')[1].split('```')[0].strip()
    # new_gen_multi_plot = new_gen_multi_plot.replace("（","(").replace("）",")").replace("：",":")

    new_gen_multi_plot = filter_plot_end(res_gen_multi)
    new_gen_multi_list = new_gen_multi_plot.split('\n')

    plot_sum_last = 0
    plot_temp_list = []
    end_temp_list = []

    for i in range(len(new_gen_multi_list)):
        item = new_gen_multi_list[i]
        item_dict, plot_num, plot_sum_last = extra_plot_end(item,plot_sum_last)
        if plot_num>-1:
            if item_dict["node_type"] == "plot":
                plot_temp_list.append(item_dict)
                pre_plot_dict[int(plot_num)] = item_dict["from_plot"]
            elif item_dict["node_type"] == "end":
                end_temp_list.append(item_dict)
                pre_plot_dict[plot_sum_last + int(plot_num)] = item_dict["from_plot"]
            else:
                print(f"Warning, no correct node_type :", item_dict["node_type"])
    # print(json.dumps(pre_plot_dict,indent=2,ensure_ascii=False))

    # 这里要看一下 gpt的输出序号不一定连续
    if not len(plot_temp_list)==plot_sum_last:
        print("剧情数量和序号未对齐")
        print(f"剧情总数: {plot_temp_list}, 剧情序号最大值: {plot_sum_last}")
    


    #print(plot_sum_last)
    multi_plot_dict = {}
    for i in range(len(plot_temp_list)):
        plot_temp = plot_temp_list[i]
        multi_plot_dict[plot_temp["plot_num"]] = {"episode_key_abstract":plot_temp["episode_key_abstract"],"next_episode":[],"info":"plot"}
        if not plot_temp["from_plot"][0] == 0:
            for from_plot_num in plot_temp["from_plot"]:
                multi_plot_dict[from_plot_num]["next_episode"].append({plot_temp["plot_num"] : plot_temp["episode_key_abstract"]})
        #pass
    for i in range(len(end_temp_list)):
        plot_temp = end_temp_list[i]
        multi_plot_dict[plot_sum_last + i + 1] = {"episode_key_abstract":plot_temp["episode_key_abstract"],"next_episode":[],"info":"end"}
        if not plot_temp["from_plot"][0] == 0:
            for from_plot_num in plot_temp["from_plot"]:
                multi_plot_dict[from_plot_num]["next_episode"].append({plot_temp["end_num"]+plot_sum_last : plot_temp["episode_key_abstract"]})
            #multi_plot_dict[plot_temp["from_plot"]]["next_episode"].append({plot_temp["end_num"]+plot_sum_last : plot_temp["episode_key_abstract"]})



    # 这里gpt输出plot不一定是plot
    for key in multi_plot_dict:
        if multi_plot_dict[key]["next_episode"] == [] and multi_plot_dict[key]["info"] == "plot":
            multi_plot_dict[key]["info"] = "end"



    # 通过bfs记录所有节点list
    story_states = []
    states_temp = [[1]]         # 记录第一个节点必定是1
    #temp = [1]

    
    while states_temp:
        temp = states_temp.pop()

        # 终止判断
        if multi_plot_dict[temp[-1]]["info"] == "end":
            story_states.append(temp)
            #print("end:", temp)
            continue
        next_episodes = multi_plot_dict[temp[-1]]["next_episode"] # list,每个list是一个dict
        next_episodes_list = [int(next(iter(item))) for item in next_episodes]
        #print(temp,next_episodes_list)

        # 记录所有的新节点
        for next_episode in next_episodes_list:
            temp_new = copy.deepcopy(temp)
            temp_new.append(next_episode)
            states_temp.append(temp_new)
            #print(temp_new)
        #temp = [temp[0]+1]
        #break

    story_dict["Settings"]["states"] = story_states
    #print(json.dumps(plot_temp_list,indent=2, ensure_ascii=False))
    #print(json.dumps(end_temp_list,indent=2, ensure_ascii=False))
    print(json.dumps(multi_plot_dict,indent=2, ensure_ascii=False))


# 单线剧情重组织，反向索引
elif if_single_plot:
    single_plot_dict = {}
    pre_plot_dict = {}
    pre_plot_dict[1] = 0
    for i in range(len(main_plot_list_re)-1):
        single_plot_dict[i+1] = {"episode_key_abstract":main_plot_list_re[i],"next_episode":[{i+2:main_plot_list_re[i+1]}],"info": "plot"}
        pre_plot_dict[i+2] = [i+1]
        #print(main_plot_list["main"][i])
    
    # 补充结局节点
    single_plot_dict[len(main_plot_list_re)] = {"episode_key_abstract":main_plot_list_re[len(main_plot_list_re)-1],"next_episode":[],"info": "end"}
    # 顺序记录所有节点
    story_dict["Settings"]["states"] = [[i+1 for i in range(len(main_plot_list_re))]]
    print(json.dumps(single_plot_dict,indent=2, ensure_ascii=False))    

if not len(story_dict["Settings"]["states"]) == len(end_temp_list):
    print("Warning, plot_tree and end_num are not matched")
print("剧情树: ",json.dumps(story_dict["Settings"]["states"],indent=2,ensure_ascii=False))
print("前一个节点的dict: ",pre_plot_dict)


{
  "1": {
    "episode_key_abstract": "收到求救信:伊森收到失踪三年的妻子米娅的求救信，前往路易斯安那州的贝克家族庄园寻找她。",
    "next_episode": [
      {
        "2": "进入庄园:伊森进入贝克家族庄园，发现庄园荒废且充满恐怖氛围，并找到被囚禁的米娅。"
      }
    ],
    "info": "plot"
  },
  "2": {
    "episode_key_abstract": "进入庄园:伊森进入贝克家族庄园，发现庄园荒废且充满恐怖氛围，并找到被囚禁的米娅。",
    "next_episode": [
      {
        "3": "米娅的变化:伊森试图带米娅逃离，但米娅突然变得狂暴并攻击伊森，伊森被迫自卫。"
      }
    ],
    "info": "plot"
  },
  "3": {
    "episode_key_abstract": "米娅的变化:伊森试图带米娅逃离，但米娅突然变得狂暴并攻击伊森，伊森被迫自卫。",
    "next_episode": [
      {
        "4": "信任的裂痕:伊森在击倒米娅后选择注射镇定剂而非杀死她，发现她身上有神秘菌核。"
      },
      {
        "5": "冷血抉择:伊森选择直接杀死狂暴的米娅，从她体内取得含有加密数据的菌核。"
      }
    ],
    "info": "plot"
  },
  "4": {
    "episode_key_abstract": "信任的裂痕:伊森在击倒米娅后选择注射镇定剂而非杀死她，发现她身上有神秘菌核。",
    "next_episode": [
      {
        "6": "贝克家族:伊森被杰克·贝克捕获，目睹其超常恢复能力，被迫参与血腥的\"猫鼠游戏\"。"
      }
    ],
    "info": "plot"
  },
  "5": {
    "episode_key_abstract": "冷血抉择:伊森选择直接杀死狂暴的米娅，从她体内取得含有加密数据的菌核。",
    "next_episode": [
      {
   

In [14]:
story_dict["Settings"]["states"]

[[1, 2, 3, 5, 8, 10, 13, 18],
 [1, 2, 3, 4, 6, 7, 9, 11, 12, 15, 17],
 [1, 2, 3, 4, 6, 7, 9, 11, 12, 14, 16]]

### 生成对应具体节点（在多线剧情生成后，单线剧情跳过上一部分）

In [15]:
# 1. 生成具体plot的events，详细
prompt_plot_detail=PromptTemplate(
    template = """请根据摘要和先前剧情编写当前摘要发生的故事（详细介绍当前摘要剧情的具体情节）
要求如下：
- 延续游戏{novel_name}中的人物，剧情，背景设定，但是可能并不是游戏中实际的剧情
- 请严格按照摘要的范围，不要扩展到未提及的后续剧情。
- 采用第三人称白描手法，禁用元叙述词汇（例如"整个故事中","在游戏中","好的","以下是关于xx的详细介绍"这种令人出戏的回复）
- 与之前的剧情连贯性，没有突兀的剧情跳跃，但是不要出现重复情节和冲突
- 用{language}回答,保持回答长度在300字符左右。

该段剧情是整体剧情的{plot_num_frac}部分,该段剧情摘要如下: {plot_str}
可供参考先前的剧情如下{pre_plot}       
    """,
    input_variables=["novel_name","plot_str","plot_num_frac","language","pre_plot"],
)
# 内容形式像讲故事一样，
# 长度请控制在300，four hundred个字符左右
# 在生化危机7中，主角伊森·温特斯接到了失踪三年的妻子米娅发来的一段神秘视频。视频里，米娅看起来非常焦虑和害怕，她告诉伊森不要来找她，但同时视频中透露了她的位置。尽管米娅的话让伊森感到困惑和担忧，但他决定追寻线索，希望能找到米娅。伊森来到了视频中显示的地点——一个废弃的庄园，在那里，他发现了被一个神秘家族控制的恐怖场景，揭开了一系列惊人的秘密，同时也开始了他的生存之旅。


In [ ]:
# 单线/多线
# 生成具体剧情

if if_single_plot:  # 这个是单线的剧情, 不走上面的多线部分, 但是要重新组织
    temp_dict = single_plot_dict
else:               # 多线剧情，额外的生成内容
    temp_dict = multi_plot_dict

plot_gen_list = {k: temp_dict[k] for k in sorted(temp_dict)}
len_main_plot = len(plot_gen_list)

i = 0
for key,value in tqdm(plot_gen_list.items()):
    plot_dict_item = plot_gen_list[key]
    #plot_dict["plot_des"] = plot_dict_item["episode_key_abstract"]
    plot_num_frac = f"{key}/{len_main_plot+1}"  # 当前情节进程数
    
    # 1. 情节生成 详细情节 - detail_events
    # 前置剧情
    states_temp = []
    pre_plot = ""
    for states_i in story_dict["Settings"]["states"]:
        if int(key) in states_i:
            states_temp = states_i
            break
    if states_temp == []:
        print("Warning")
    else:
        for state in states_temp:
            # print(state,key)
            # print(states_temp)
            if not state == int(key):
                pre_plot += plot_gen_list[state]["episode_desc"] + "\n"
            else:
                break
    # chain_plot_detail = prompt_plot_detail | llm_agent
    # res_plot_detail = chain_plot_detail.invoke({"novel_name": novel_name, "plot_str": plot_dict_item["episode_key_abstract"],"plot_num_frac":plot_num_frac,"language":language,"pre_plot":pre_plot}).content

    prompt3 = prompt_plot_detail.format(novel_name=novel_name, plot_str=plot_dict_item["episode_key_abstract"],plot_num_frac=plot_num_frac,language=language,pre_plot=pre_plot)
    res_think, res_gen_plot_detail = llm_chat_analysis("gpt-4o", prompt3, "str")

    res_gen_plot_detail = res_gen_plot_detail.replace("\n"," ").strip()
    print(key,res_gen_plot_detail)
    plot_gen_list[key]["episode_desc"] = res_gen_plot_detail

    # break
    
    # gpt 5*剧情长度
 

  6%|▌         | 1/18 [00:07<02:00,  7.10s/it]

1 伊森·温特斯收到了一封来自失踪三年的妻子米娅的求救信，信中简短而急促的字句让他心中充满不安。信的落款地是路易斯安那州一个偏僻的地方——贝克家族庄园。尽管心中充满疑虑，伊森还是决定亲自前往一探究竟。他驱车穿过蜿蜒的乡村小路，最终抵达了庄园的入口。庄园四周被浓密的森林包围，显得格外阴森。伊森推开锈迹斑斑的大门，踏入了这个充满未知和危险的地方，心中只想着要找到米娅并带她安全回家。


 11%|█         | 2/18 [00:13<01:49,  6.86s/it]

2 伊森·温特斯推开贝克家族庄园的大门，迎面而来的是一股腐朽的气息。庄园内一片荒废，墙壁上爬满了藤蔓，地板发出嘎吱声，仿佛在诉说着这里曾经的辉煌与如今的凋零。伊森小心翼翼地穿过昏暗的走廊，墙上的旧照片隐约可见贝克家族的身影，令人不寒而栗。经过一番搜寻，他在地下室找到了被囚禁的米娅。她神情恍惚，身体虚弱，显然经历了非人的折磨。伊森心中一紧，急忙上前解开她的束缚，试图唤醒她的意识。然而，米娅的反应却异常冷淡，仿佛对周围的一切都充满了警惕和陌生。伊森意识到，这个庄园隐藏着更深的秘密，而他必须尽快带米娅离开这个危险之地。


 17%|█▋        | 3/18 [00:20<01:44,  6.96s/it]

...

16 伊森在摧毁主菌核后，终于迎来了短暂的平静。他在废墟中找到米娅，两人紧紧相拥，感受到久违的安全。然而，伊森清楚地知道，他们体内仍残留着菌核的痕迹，这意味着危险尚未完全解除。就在此时，政府特工赶到现场，他们迅速评估了情况，并决定协助伊森和米娅撤离庄园。特工们熟练地引导他们穿过庄园的残骸，避开仍然不稳定的区域。伊森和米娅在特工的保护下，终于走出这个充满噩梦的地方。尽管身体疲惫不堪，但他们心中充满了对未来的希望。离开庄园后，伊森和米娅坐在特工的直升机上，望着逐渐远去的庄园废墟，心中明白这场噩梦终于结束。然而，他们也深知，体内的菌核仍是一个潜在的威胁，未来的路依然充满挑战。


 94%|█████████▍| 17/18 [01:33<00:05,  5.64s/it]

17 伊森在与菌核融合后，成为了新的菌核意识体宿主。他的意识逐渐被菌核侵蚀，开始失去自我。贝克庄园在他的操控下，形成了更大的感染区，周围的植被和生物都被霉菌覆盖，成为菌核的延伸。米娅在伊森的影响下，沦为傀儡，失去了自主意识，只能机械地执行伊森的命令。庄园内的感染不断扩散，仿佛预示着一个新的纪元即将开启。伊森的内心深处仍残存着一丝理智，他努力抗争着菌核的控制，但这种斗争显得无比艰难。随着感染区的扩大，伊森意识到自己已经成为了他曾经想要摧毁的存在，而米娅的命运也与他紧密相连。


100%|██████████| 18/18 [01:37<00:00,  5.41s/it]

18 庄园在爆炸中化为废墟，伊森·温特斯为了拯救米娅，毅然决定牺牲自己。他在最后时刻将米娅推上逃生通道，而自己却被困在火海中。火焰吞噬了庄园的一切，伊森的身影逐渐消失在烈焰中。与此同时，政府迅速介入，将整个事件掩盖为意外事故，所有与生化实验相关的证据都被销毁。米娅在逃出生天后，心中充满了对伊森的思念和对真相的无奈。庄园的秘密随着爆炸化为灰烬，成为无人知晓的过往。


In [17]:
# print(json.dumps(plot_gen_list,indent=2,ensure_ascii=False))

#### story plot refine 以及 regenerate

In [ ]:

Settings = story_dict["Settings"]
story_style = Settings["story_style"]
leading_role = Settings["leading_role"]
story_desc = Settings["story_desc"]
story_goal = Settings["story_goal"]
story_chars = Settings["story_chars"]
states = Settings["states"]
story_language = Settings["story_language"]

# score and refine
# 原因修改一体化, 未启用
prompt_evaluate_EpiSeq=PromptTemplate(
    template = """作为故事分析师, 请根据以下标准严格评估剧情树合理性, 严格寻找问题与不足，展示思考过程。若发现不合格章节, 请输出包含问题章节的JSON对象, 说明不合理的原因，改进建议以及改进后的章节内容(只需指出不合理的章节) 

# 评估标准
1. 连贯性: 整体章节必须连贯, 相邻章节需保持时间/逻辑连续性，内容不重叠不跳跃; 相邻的章节剧情应该连续但不相交，不要和相邻的章节重叠
2. 合理性: 剧情必须形成完整闭环, 前后剧情逻辑合理; 符合世界观以及人物性格;
3. 唯一性: 不同章节不能重复, 需保持唯一（含非相邻章节）
4. 完整性: 剧情完整; 没有节奏问题/突兀转折/不自然中断
5. 吸引性: 整体故事是否有趣(例如: 起承转合, )
6. 叙述视角: 每个章节必须严格以主角[{leading_role}]视角展开
 - 注意: 剧情树的序号可能不连贯, 仅是作为剧情的唯一标识, 不作为连贯性的评判标准, 无需修改, 只需关注序号后的实际剧情

# 修改约束
1. 严禁删除/新增章节！仅可修改现有章节内容
2. 章节剧情冲突/重复时优先修改后续章节
3. 你改进后的章节需要注意是否与其他章节或者新改写的章节冲突或者重复
4. 对故事进行修改时，应保持其整体风格, 基调, 人物整体命运不变
5. 如果你建议修改结局, 那么你改进的结局需与其他存在的结局有本质差异。    

# 输出格式(json):
{{"(章节序号)":{{
        "存在问题": "(你的思考过程，不合理的原因)",
        "改进建议": "(如何修改这部分章节)",
        "改进后章节": "(改进后的章节内容)",
    }},
    ...(多个有问题的章节)
}} 

# 上下文信息
- 故事风格: {story_style}
- 世界观设定: {story_desc}
- 核心目标: {story_goal}
- 人物信息: {story_chars}
- 其他已经存在的结局: {other_ans}
- 当前剧情树: {episode_seq}

请用{language}输出JSON，确保语法正确    
    """,
    input_variables=["leading_role","language","episode_seq","story_style","story_desc","story_goal","story_chars","other_ans"],
)

 

# eva_score
prompt_Eva_Score_Epi = PromptTemplate(
    template = """你作为故事分析师, 请根据以下标准严格评估剧情树, 严格寻找问题与不足, 尽可能严厉, 给出专业评分(0-100整型数), 评分须严格遵循标准，扣分必须注明具体原因。

### 评分维度及权重(总分100)
  **基础结构(20分)**: 
    - 每个章节是否以主角[{leading_role}]视角展开
    - 主角的核心目标是否明确且贯穿始终
    - 不同章节发生事件是否保持唯一(含非相邻章节), 没有重复事件
    - 时间轴是否严格单向流动(无倒叙/插叙)
  **连贯性(20分)**: 
    - 整体章节剧情是否连贯
    - 相邻章节是否保持时间/逻辑连续性, 剧情不跳跃
    - 相邻章节是否没有重叠内容(相邻的章节剧情应该连续但不相交)
    - 非相邻章节是否通过伏笔或线索形成隐性关联  
  **合理性(20分)**: 
    - 剧情整体是否形成完整闭环
    - 前后剧情逻辑是否合理, 事件发生是否自然; 
    - 剧情是否存在矛盾或者发生重复事件
    - 剧情是否符合世界观以及对应人物性格/行为
  **完整性(20分)**: 
    - 整体剧情是否完整(例如: 后期突兀出现没有提到的关键道具/人物; 结局结束过于突然)
    - 有无节奏问题(例如: 拖沓/突兀转折/不自然中断)
    - 前期剧情是否对后期剧情有所影响(例如: 前期细节/道具在后期转化为关键线索/发生作用/反转, 首尾呼应)
  **吸引性(20分)**: 
    - 整体故事是否有趣/引人入胜
    - 整体剧情是否节奏合理(例如: 起承转合)
    - 是否有引人入胜的转折点,悬念和高潮
  **注意**: 剧情树的序号可能不连贯, 仅是作为剧情的唯一标识, 不作为连贯性的评判标准, 无需修改, 只需关注序号后的实际剧情

### 输出格式
返回一个json格式的字典,包含两个键, 评分的原因以及最终的具体分数
{{
    "reason": "(各个指标的（评分/总分）以及扣分原因)"
    "score": 最终评分(仅数字)
}}

### 上下文信息
- 故事风格: {story_style}
- 世界观设定: {story_desc}
- 核心目标: {story_goal}
- 人物信息: {story_chars}
- 当前剧情树: {episode_seq}   
    """,
    input_variables=["leading_role","episode_seq","story_style","story_desc","story_goal","story_chars"],
)

# refine_score
prompt_Eva_Opt_Epi=PromptTemplate(
    template = """你作为专业作家, 接下来我将给你一个剧情大纲评价, 请根据剧情大纲以及评价对你剧情的不足之处进行修改
首先根据总体大纲评价找到不合理的章节, 若发现不合格章节, 请输出包含问题章节的JSON对象, 说明对应具体章节的不合理原因，改进建议以及改进后的章节内容(只需指出和改进不合理的章节) 

# 修改约束
1. 严禁删除/新增章节！仅可修改现有章节内容
2. 章节剧情冲突/重复/逻辑矛盾时优先修改后续章节
3. 如果你建议修改结局/最后一个章节, 那么你改进的结局需与其他存在的结局有本质差异。 
4. 无冲突/重复: 修改后前后剧情一致, 不冲突不重复; 修改后的剧情是否符合人物行为性格和世界观; 
5. 剧情合理: 对故事进行修改时，应保持其整体风格, 基调, 人物整体命运不变; 后期关键道具/人物应该提前铺垫
6. 剧情连贯: 相邻的章节剧情应该连续但不相交, 内容不重叠不跳跃; 时间轴严格单向流动(无倒叙/插叙); 
7. 剧情主线一致(以主角[{leading_role}]视角展开)
 - 注意: 剧情树的序号可能不连贯, 仅是作为剧情的唯一标识, 不作为连贯性的评判标准, 无需修改, 只需关注序号后的实际剧情

# 输出格式(json):
{{"(章节序号)":{{
        "存在问题": "(对应章节的不合理原因)",
        "改进建议": "(如何修改这部分章节)",
        "改进后章节": "(改进后的章节内容)",
    }},
    ...(多个有问题的章节)
}} 

# 上下文信息
- 故事风格: {story_style}
- 世界观设定: {story_desc}
- 核心目标: {story_goal}
- 人物信息: {story_chars}
- 其他已经存在的结局: {other_ans}
- 当前剧情大纲: {episode_seq}
- 剧情大纲的不足之处: {outline_refine_reason}

请用{language}输出JSON，确保语法正确    
    """,
    input_variables=["leading_role","language","outline_refine_reason","episode_seq","story_style","story_desc","story_goal","story_chars","other_ans"],
)

In [36]:
print(json.dumps(states,indent=2,ensure_ascii=False))

[
  {
    "seq": [
      1,
      2,
      3,
      5,
      8,
      10,
      13,
      18
    ],
    "Eva_outline_ori_think": "主线结构(10/10): 所有章节均以伊森视角展开，核心目标明确。连贯性(12/20): Episode 5中菌核的引入缺乏过渡，卢卡斯的首次出现未铺垫，导致逻辑跳跃；自毁程序与卢卡斯动机的衔接生硬。合理性(10/20): 卢卡斯的背叛动机不充分，菌核的加密数据未在后期发挥实质作用，削弱了剧情逻辑。唯一性(10/10): 无重复剧情。完整性(12/20): 政府掩盖结局缺乏伏笔，菌核的关键性未充分体现，削弱闭环结构。吸引性(5/10): 高潮分布不均，卢卡斯合作到背叛的转折过于突兀，影响节奏。",
    "Eva_outline_ori_score": 59,
    "Eva_outline_refine_think": "主线结构(10/10): 所有章节均以伊森视角展开，核心目标明确贯穿始终。连贯性(15/20): 相邻章节存在跳跃（如ep3到ep5缺乏菌核发现的过渡，ep10到ep13时间衔接模糊），但非相邻章节通过菌核数据、军方伏笔形成隐性关联。合理性(15/20): 米娅恢复过程未明确解释，卢卡斯动机合理性存疑，军方介入铺垫不足。唯一性(10/10): 各章节剧情无重复。完整性(15/20): 结局伊森生死未明，防化服人员出现突兀，数据备份舱伏笔未完全回收。吸引性(8/10): 高潮分布合理但部分转折生硬（如卢卡斯突然合作）。",
    "Eva_outline_refine_score": 73
  },
  [
    1,
    2,
    3,
    4,
    6,
    7,
    9,
    11,
    12,
    15,
    17
  ],
  [
    1,
    2,
    3,
    4,
    6,
    7,
    9,
    11,
    12,
    14,
    16
  ]
]


In [49]:

other_ans = ""
other_ans_num = []

# 评估连贯性
for j in tqdm(range(len(states))):
    state = {}
    # print(type(states[j]))
    # print(json.dumps(states[j],indent=2,ensure_ascii=False))
    if type(states[j]) == dict:
        
        state["seq"] = states[j]["seq"]
    elif type(states[j]) == list:
        state["seq"] = states[j]
    else:
        print("Warning, wrong type")

    episode_seq = ""
    episode_seq_refine = ""
    # 剧情序列
    # print(state)

    episode_seq += "\n".join(f"(episode {k}) {plot_gen_list[k]['episode_desc']}\n" for k in state["seq"])
    other_ans_num.append(state["seq"][-1]) if not state["seq"][-1] in other_ans_num else None

    # 判断总体章节信息
    # chain_evaluate_EpiSeq = prompt_evaluate_EpiSeq | llm_agent
    # res_evaluate_EpiSeq = chain_evaluate_EpiSeq.invoke({"leading_role":leading_role,"language":story_language,"episode_seq":episode_seq,"story_style":story_style,"story_desc":story_desc,"story_goal":story_goal,"story_chars":story_chars,"other_ans":other_ans}).content
    # cleaned_json_str1 = res_evaluate_EpiSeq.split('```json')[1].split('```')[0].strip()
    # res1_dict = {}
    # res1_dict = json.loads(cleaned_json_str1)

    prompt4_1 = prompt_Eva_Score_Epi.format(leading_role=leading_role,episode_seq=episode_seq,story_style=story_style,story_desc=story_desc,story_goal=story_goal,story_chars=story_chars)
    res_Eva_think_1, res_Eva_Score_Epi1 = llm_chat_analysis("gpt-4o", prompt4_1, "dict")

    # print(res_Eva_think_1)
    # print(json.dumps(res_Eva_Score_Epi1,indent=2,ensure_ascii=False))
    state["Eva_outline_ori_think"] = res_Eva_Score_Epi1["reason"]
    res_Eva_Score_Epi1["score"] = int(res_Eva_Score_Epi1["score"])
    state["Eva_outline_ori_score"] = res_Eva_Score_Epi1["score"]
    # break
    if res_Eva_Score_Epi1["score"] < 85:
        prompt4_2 = prompt_Eva_Opt_Epi.format(leading_role=leading_role,language=story_language,outline_refine_reason=res_Eva_Score_Epi1["reason"],episode_seq=episode_seq,story_style=story_style,story_desc=story_desc,story_goal=story_goal,story_chars=story_chars,other_ans=other_ans)
        res_think, res_Eva_Opt_Epi = llm_chat_analysis("deepseek-r1", prompt4_2, "dict")
        
        if not state["seq"][-1] in other_ans_num:
            other_ans_num.append(state["seq"][-1])
            other_ans += "Other end: " + plot_gen_list[state["seq"][-1]]["episode_desc"]

        # print(episode_seq)
        for key in res_Eva_Opt_Epi:
            pass
            key_num = key
            
            if type(key) == str:
                key_num = numbers = re.findall(r'\d+', key_num)
                key_num = [int(num) for num in key_num][0]

            # print(key_num,plot_gen_list[int(key_num)]["episode_desc"])
            # print(json.dumps(res_Eva_Opt_Epi[key],indent=2,ensure_ascii=False))

            plot_gen_list[int(key_num)]["episode_desc_old"] = plot_gen_list[int(key_num)]["episode_desc"]
            plot_gen_list[int(key_num)]["episode_desc"] = res_Eva_Opt_Epi[key]["改进后章节"]
            plot_gen_list[int(key_num)]["episode_old_question"] = res_Eva_Opt_Epi[key]["存在问题"] + " " + res_Eva_Opt_Epi[key]["改进建议"]
        
            episode_seq_refine += "\n".join(f"(episode {k}) {plot_gen_list[k]['episode_desc']}\n" for k in state["seq"])
            prompt4_3 = prompt_Eva_Score_Epi.format(leading_role=leading_role,episode_seq=episode_seq_refine,story_style=story_style,story_desc=story_desc,story_goal=story_goal,story_chars=story_chars)
            res_Eva_think_2, res_Eva_Score_Epi2 = llm_chat_analysis("gpt-4o", prompt4_3, "dict")

            # print(res_Eva_think_2)
            # print(json.dumps(res_Eva_Score_Epi2,indent=2,ensure_ascii=False))
            state["Eva_outline_refine_think"] = res_Eva_Score_Epi2["reason"]
            res_Eva_Score_Epi2["score"] = int(res_Eva_Score_Epi2["score"])
            state["Eva_outline_refine_score"] = res_Eva_Score_Epi2["score"]

            break

    story_dict["Settings"]["states"][j] = state
    
    # break

# print(res_think)
# print(json.dumps(res_evaluate_EpiSeq,indent=2,ensure_ascii=False))

#story_dict_list_new[i] = story_dict_new

# 一轮deepseek耗时 200-240s

100%|██████████| 3/3 [00:15<00:00,  5.01s/it]


In [50]:
print(json.dumps(res_Eva_Score_Epi1,indent=2,ensure_ascii=False))
print(json.dumps(res_Eva_Opt_Epi,indent=2,ensure_ascii=False))
print(json.dumps(res_Eva_Score_Epi2,indent=2,ensure_ascii=False))

{
  "reason": "1. 主线结构(10/10): 整个剧情树都以伊森·温特斯的视角展开，主角的核心目标明确且贯穿始终，即寻找并拯救妻子米娅，同时揭开庄园的秘密。2. 唯一性(10/10): 每个章节的剧情都各具特色，没有重复的情节。3. 连贯性(18/20): 整体章节连贯，时间和逻辑上大致连续，但episode 4和episode 6之间的过渡稍显突兀，缺少连接情节。4. 合理性(18/20): 整体剧情形成闭环，逻辑合理，符合世界观和人物性格，但episode 11中关于血清的设定略显复杂，可能导致理解上的困难。5. 完整性(18/20): 剧情完整，前期的设定在后期得到了呼应，但episode 12的幻象情节虽然增加了心理冲击，但对主线影响不大，显得有些多余。6. 吸引性(18/20): 故事节奏紧凑，悬念和转折合理分布，但在某些情节上（如episode 14的菌核爆炸）可以更具戏剧性。",
  "score": 92
}
{
  "4": {
    "存在问题": "镇定剂来源缺乏合理性",
    "改进建议": "在解救米娅的场景中增加医疗箱细节，解释镇定剂来源",
    "改进后章节": "伊森将米娅制服后，在囚室角落发现锈迹斑斑的医疗箱。他不忍心伤害妻子，用箱内发现的镇定剂完成注射。米娅逐渐安静时，伊森注意到她后颈浮现的菌核正随脉搏跳动，这个发现让他想起三年前米娅在生物公司的工作经历。墙外突然响起的金属拖拽声迫使伊森停止思考，他必须立即行动。"
  },
  "11": {
    "存在问题": "与第九章菌核网络探索重复且选择逻辑断裂",
    "改进建议": "将血清配方与菌核网络结合，强化选择代价的戏剧冲突",
    "改进后章节": "伊森在菌核洞穴发现岩壁上嵌着数支蓝色血清。玛格丽特的声音突然在洞穴回荡：「这些血清能暂时抑制感染，但每次使用都会加速菌核增殖」。伊森颤抖着触碰试管，发现每支血清都连接着菌丝脉络——取出即会触发防御机制。此刻洞穴剧烈震动，菌核深处传来米娅的呼救声，伊森必须在拯救妻子与保全解药间做出抉择。"
  },
  "17": {
    "存在问题": "宿主转化与米娅傀儡化过程缺乏铺垫",
    "改进建议": "通过菌核意识侵蚀表现渐变过程，保留人性挣扎的细节",
    "改进后章节": "伊

In [ ]:
print(json.dumps(plot_gen_list,indent =2 ,ensure_ascii=False))

{
  "1": {
    "episode_key_abstract": "收到消息:伊森·温特斯收到失踪三年的妻子米娅的录像，前往路易斯安那州的贝克家寻找她。",
    "next_episode": [
      {
        "2": "贝克家探险:伊森进入贝克家，发现房子被感染，遭遇疯狂的贝克家族成员。"
      }
    ],
    "info": "plot",
    "episode_desc": "伊森·温特斯在一个平常的夜晚，收到了一封来自失踪三年的妻子米娅的录像。录像中，米娅神情紧张，似乎身处险境，只留下一个模糊的地址。伊森心急如焚，决定不顾一切前往路易斯安那州的贝克家寻找她。他驱车穿越幽暗的森林，心中充满了不安和期待。贝克家的大门在夜色中显得格外阴森，伊森推开生锈的铁门，踏入这片未知的领域，决心揭开妻子失踪的谜团。"
  },
  "2": {
    "episode_key_abstract": "贝克家探险:伊森进入贝克家，发现房子被感染，遭遇疯狂的贝克家族成员。",
    "next_episode": [
      {
        "3": "重聚米娅:伊森找到米娅，但她被感染并攻击伊森，伊森被迫自卫。"
      }
    ],
    "info": "plot",
    "episode_desc": "伊森·温特斯穿过贝克家阴森的庭院，进入了那座破败的老宅。空气中弥漫着腐朽的气息，墙壁上斑驳的痕迹诉说着岁月的侵蚀。房间内昏暗的灯光摇曳不定，仿佛在警示他未知的危险。伊森小心翼翼地穿过走廊，突然听到一阵低沉的脚步声。他屏住呼吸，躲在一扇半掩的门后，窥探着声音的来源。一个身材魁梧的男人出现在视线中，脸上挂着疯狂的笑容，正是贝克家族的杰克。伊森心跳加速，意识到自己必须小心行事，才能在这个充满危险的地方找到米娅。他继续在房子中探索，试图避开杰克的追踪，同时寻找任何可能的线索，揭开这座被感染的房子背后的秘密。"
  },
  "3": {
    "episode_key_abstract": "重聚米娅:伊森找到米娅，但她被感染并攻击伊森，伊森被迫自卫。",
    "next_episode": [
      {
        "4": "逃离与追捕:伊森试图逃离贝克家，但被杰克·贝克抓住并囚

### plot其他内容生成

In [ ]:

# 2. 其他信息
prompt_plot_info=PromptTemplate(
    template = """请用{language}整理「{novel_name}」游戏中的某段plot中的以下内容
plot摘要如下:{plot_str}
(该段剧情是整体剧情的{plot_num_frac}部分)
# 整理内容
▶ target: 以主角「{story_leading_role}」视角说明本段plot的核心目的(仅限目的，不涉及本段剧情实际结果)
▶ scene: 本段plot的背景和它发生的地点/时间/环境等, 需要结合当前plot中即将发生的剧情进行介绍,但是不要涉及到当前plot发生的内容
▶ role: 本段plot中出现的主要人物, 需要介绍性格与当前情节的状态等关键信息, 人物数量小于5个

# 格式规范
 - 使用如下格式`**:详细内容`, 保持结果的格式和例子中的格式一致
 - 输出以json格式: '{{key:value}}(键值对的形式dict) '重构
 - 禁止使用「玩家」「游戏」「剧情中」等出戏表述
 - 保持叙事内视角的一致性
        
# 示例结构
{{
    "target":"劳拉的主要目标是荒岛上生存下来，找到并解救她的团队成员，同时探索荒岛。",
    "scene":"劳拉和她的团队在寻找邪马台古国的途中遭遇风暴，船只失事，劳拉漂流到一个未知的岛屿上。这个岛屿是日本以东、福尔摩沙龙三角中的一座小岛——邪马台。那座岛屿以及曾经存在于岛上的王国充满着神秘的传说，许多飞机和船只都曾在附近遭遇狂风暴雨而失事，残骸布满着它的海岸，因而闻名。",
    "role":[
        "劳拉·克劳馥: 幼稚单纯且充满自由精神和荒野情怀。此时劳拉还只是一个21岁的刚踏出校园的小女孩。对亡故的父亲的不解和对冒险的渴望促使劳拉和志同道和的同窗好友珊曼莎一起乘上了驶往邪马台古国的坚忍号，在寻找邪马台古国的途中，劳拉所在的船只遭遇风暴失事，她漂流到一个未知的岛屿上，孤身一人在岛上醒来，被一个野蛮人从背后袭击。醒来后发现自己被吊在一个类似祭台的环境中，周围满是尸骨，开始她的求生之旅。",
        "康拉德·罗斯: 睿智，理性，坚韧而一丝不苟。坚忍号探险船的船长，52岁的资深冒险家，曾在皇家海军陆战队突击旅服役。他与劳拉的父母私交颇深，对队员们的安危非常关心，是劳拉的老师和精神领导者。在风暴中与劳拉一同遭遇船难，之后在岛上与劳拉重逢，共同面对岛上的危险。",
        "珊曼莎·西村: 大大咧咧，习惯过奢侈阔绰的生活。实习纪录片制片人，也是劳拉的同窗好友。她拥有葡萄牙和日本的双重国籍，家境殷实。珊曼莎是太阳女王卑弥呼的后裔，珊曼莎下落不明，但是的背包被劳拉找到，里面有一台DV、火柴和对讲机。",
    ]
}}""",
    input_variables=["novel_name","plot_str","plot_num_frac","story_leading_role","language"],
)
# 并以json格式: 'main:[xx:xx, xx:xx, ...] '重构。
#event:船只失事后，劳拉醒来发现自己孤身一人在岛上，她必须适应野外生存，学会使用岛上的资源来维持生命，同时寻找她的团队成员。

#探险队所在的“坚韧号”上，劳拉和久负盛名的考古学家惠特曼博士争吵着船的航向，劳拉推测邪马台大致位于日本东部的龙三角海域，这片海域如同百慕大魔鬼三角一样，海难频发，是个名副其实的“恶魔海”。虽然工程师艾利克斯给出了龙三角非同寻常的气象图，但船长兼“老父亲”的罗斯还是遵循劳拉的意见将船驶向了龙三角。不出意外，强烈的风暴袭击了船只，巨大的雷电直接将船劈为两段，劳拉也在暴风中与其他成员失散，飘到了一处未知的沙滩上，正呼喊着队友名字时，一个野蛮人从背后袭击了她。醒来后，劳拉 发现自己被吊在一个如同祭台的环境内，周围满是尸骨，她通过晃动用祭台的火烧断绳子，成功挣脱束缚，却被下方一根生锈的钢钉刺穿左腹。求生欲的驱使下，劳拉强忍剧痛硬生生拔出了钢钉，摇摇晃晃的向前走去。穿过可怖的祭台和阴暗的地道，劳拉最终逃出了山洞。拖着受伤的身躯，劳拉沿着海岸的山路向前走去，在珊曼莎的背包中找到一台DV、火柴和对讲机，靠着火柴升起了火堆，而一天的逃生让她疲惫不堪，不久便沉沉睡去。第二天清晨，饥饿难耐的劳拉回忆着罗斯教给她的生存经验射杀了一头野鹿。


# 生成两个节点的过渡句，不好，不像是triggers而是跳转，如果后面需要过渡句似乎还可以

# # 未使用
# prompt_plot_transitions=PromptTemplate(
#     template = """请编写在{novel_name}游戏两段plot中间的过渡情节的摘要,接下来我会给你一个当前情节，和一个将要发生的情节，用{language}
#         请详细写出跳转到下一个故事节点的关键原因，需要合理并且符合人物行为
#         内容需要与当前情节的主要目标"{plot_target}"以及下一段剧情有所关联，应该是当前plot与下一段plot之间的内容，需要做到承上启下
#         不要提到"玩家","在xx游戏中"，"在剧情中"等令人出戏的字眼，在人物介绍时不要提到游戏名称

#         该段剧情是整体剧情的{plot_num_frac}部分,plot摘要如下:{plot_str}
#         下一段剧情是整体剧情的{next_episode_num_frac}部分,plot摘要如下:{next_episode_str}
#         像讲故事一样直接介绍,精简输出，150字以内
#     """,
#     input_variables=["novel_name","plot_str","next_episode_str","plot_target","plot_num_frac","next_episode_num_frac","story_leading_role","language"],
# )


# 3. triggers
# 生成两个triggers, 关键在于不要像过渡
# 意图要明确，字数别太多
# 类似于条件语句：if xxx then xxx
prompt_plot_triggers=PromptTemplate(
    template = """基于「{novel_name}」世界观，用{language}分析从当前剧情向下一情节过渡条件(trigger)
# 核心要求
 - 使用<IF>(标注情节转换的关键原因)和<THEN>(标注触发原因后产生的即时结果)作为标识符; <IF>必须直接关联当前核心目标「{plot_target}」, <THEN>必须自然衔接下一情节
 - 简洁明了地阐述从当前情节跳转到下一个情节的关键原因, 如果有其他过渡条件, 当前跳转原因应该与其有较大差异
 - 人物动机需符合性格设定, 事件发展需符合世界观规则; 采用「内因驱动>外因推动」的优先级
 - 避免提及「玩家」「游戏」「剧情中」元叙事词汇
   
# 格式示例
<IF> **在当前剧情中跳转需要达成的条件** <THEN> **达成条件后触发的关键事件(事件承上启下，发生于下一段剧情之前)，导致剧情向下一个情节发展**(请根据实际情况填写**中的内容)

# 上下文内容
 - 该段剧情是整体剧情的{plot_num_frac}部分
 - 当前plot摘要:{plot_str}
 - 待衔接plot摘要:{next_episode_str}
 - 其他过渡条件如下: {else_triggers}
 - 故事风格: {story_style}; 世界观: {story_desc}; 主要人物: {story_chars}
    """,
    input_variables=["novel_name","plot_str","next_episode_str","plot_target","plot_num_frac","else_triggers","story_style","story_desc","story_chars","language"],
)


In [ ]:
   
story_dict["Episodes"] = []

for key,value in tqdm(plot_gen_list.items()):

    plot_dict_item = plot_gen_list[key]

    plot_dict = copy.deepcopy(template_dict_plot)
    plot_dict["episode_id"] = key
    plot_dict["pre_episode_id"] = pre_plot_dict[key]
    plot_dict["episode_desc"] = plot_dict_item["episode_desc"]
    plot_dict["episdoe_desc_brief"] = plot_dict_item["episode_key_abstract"]
    
    # 2. 其他故事信息  - target,scene,role
    # chain_plot_info = prompt_plot_info | llm_agent
    # res_plot_info = chain_plot_info.invoke({"novel_name": novel_name, "plot_str": plot_dict_item["episode_key_abstract"],"plot_num_frac":plot_num_frac,"story_leading_role":story_leading_role,"language":language}).content
    
    plot_num_frac = f"{key}/{len_main_plot+1}"  # 当前情节进程数
    res_gen_plot_info = {"target":"","scene":"","role":{}}
    prompt4 = prompt_plot_info.format(novel_name=novel_name, plot_str=plot_dict_item["episode_key_abstract"],plot_num_frac=plot_num_frac,language=language,story_leading_role=story_leading_role,story_style= story_style,story_desc = story_desc,story_chars= story_chars)
    res_think, res_gen_plot_info = llm_chat_analysis("gpt-4o", prompt4, "dict")
    # print(json.dumps(res_gen_plot_info,indent=2,ensure_ascii=False))

    plot_dict["episode_goal"] = res_gen_plot_info["target"]
    plot_dict["episode_scene"] = res_gen_plot_info["scene"]

    episode_role = res_gen_plot_info["role"]
    if type(episode_role) == dict:
        plot_dict["episode_chars"] = episode_role
    elif type(episode_role) == list:
        epi_role_dict = {}
        for role_item in episode_role:
            if type(role_item) == str:
                role_item = role_item.replace("：",":")
                epi_role_dict[role_item.split(":")[0].strip()] = role_item.split(":")[1].strip()
            else:
                print(f"warning, episode_role type is list[{type(role_item)}]")
                print("llm能力应该不会有这种问题")
        plot_dict["episode_chars"] = episode_role
    else:
        print(f"warning, episode_role type is {type(episode_role)}")
        print("llm能力应该不会有这种问题")
    #"""
    # 3. 跳转情节生成 - triggers
    # 需要当前情节的目标
    plot_target = plot_dict["episode_goal"]

    if plot_dict_item["next_episode"] == []:
        plot_dict["triggers"] = [{}]
    else:
        plot_dict["triggers"] = []
        # 对于每个trigger生成？
        # 但是不确定这样会不会与不同的选项冲突

        # 将生成过的triggers记录一下
        else_triggers = ""

        for item in plot_dict_item["next_episode"]:
            next_key = list(item.keys())[0]
            next_value = item[next_key]

            
            trigger_plot = {"condition":"","next_episode": next_key}
            next_episode_num_frac = f"{next_key}/{len_main_plot+1}"
            
            # chain_plot_triggers = prompt_plot_triggers | llm_agent
            # #res_plot_triggers = chain_plot_triggers.invoke({"novel_name": novel_name, "plot_str": plot_dict_item["episode_key_abstract"], "next_episode_str": next_value, "plot_target" : plot_target,"plot_num_frac":plot_num_frac,"next_episode_num_frac":next_episode_num_frac,"story_leading_role":story_leading_role,"language":language}).content
            # res_plot_triggers = chain_plot_triggers.invoke({"novel_name": novel_name, "plot_str": plot_dict_item["episode_key_abstract"], "next_episode_str": next_value, "plot_target" : plot_target,"plot_num_frac":plot_num_frac,"else_triggers":else_triggers,"story_leading_role":story_leading_role,"language":language}).content

            prompt5 = prompt_plot_triggers.format(novel_name=novel_name, plot_str=plot_dict_item["episode_key_abstract"], next_episode_str=next_value, plot_target=plot_target, plot_num_frac=plot_num_frac, else_triggers=else_triggers, language=language)

            res_think, res_gen_plot_triggers = llm_chat_analysis("gpt-4o", prompt5, "str")
            # print(json.dumps(res_gen_plot_triggers,indent=2,ensure_ascii=False))

            if else_triggers == "":
                else_triggers = """
需要注意的是, 在这段剧情中, 除了你现在需要编写的trigger以外, 还存在其他triggers, 它们可以导致当前情节跳转到不同的分支。
你需要确保你编写的触发条件与其他触发条件有显著差别，以确保剧情能够清晰地沿着不同的路径发展
具体来说, <IF>之后的条件和<THEN>之后触发的剧情都应与其他triggers的<IF>和<THEN>的内容产生较大的不同
以下是这个情节中的其他triggers: \n
                """
            else_triggers += res_gen_plot_triggers + "\n"

            #print("当前剧情:\n",plot_dict_item["episode_key_abstract"])
            #print(f"过渡剧情:\n{res_plot_triggers}")
            #print(f"下一个剧情:\n{next_value}")
            trigger_plot["condition"] = res_gen_plot_triggers
            plot_dict["triggers"].append(trigger_plot)
    #print("3")
    #break

    #print(json.dumps(plot_dict,indent=2, ensure_ascii=False))  
    story_dict["Episodes"].append(plot_dict)
    
    #break

    # 其他故事

100%|██████████| 19/19 [45:06<00:00, 142.44s/it]  


In [ ]:
print(json.dumps(story_dict,indent=2,ensure_ascii=False))

{
  "Settings": {
    "story_id": "",
    "story_name": "生化危机7(Resident Evil 7)",
    "story_style": [
      "恐怖生存",
      "心理惊悚",
      "黑暗悬疑",
      "紧张刺激",
      "超自然元素",
      "家庭悲剧",
      "孤立无援"
    ],
    "leading_role": "伊森·温特斯",
    "story_desc": "故事发生在2017年，地点位于美国路易斯安那州的一个偏远农场，主角伊森·温特斯接到失踪三年的妻子米娅的神秘信息后，前往贝克家族的庄园寻找她。贝克家族被一种神秘的生物感染，变得极其危险和疯狂。伊森在探索过程中揭开了关于贝克家族和感染源的恐怖秘密。",
    "story_goal": "伊森的主要目标是找到并营救他的妻子米娅，同时揭开贝克家族背后隐藏的真相，并设法逃离这个充满危险的庄园。在这个过程中，他需要面对被感染的贝克家族成员和其他恐怖生物的威胁。",
    "story_chars": {},
    "storylines": [
      "收到消息:伊森·温特斯收到失踪三年的妻子米娅的录像，前往路易斯安那州的贝克家寻找她。",
      "贝克家探险:伊森进入贝克家，发现房子被感染，遭遇疯狂的贝克家族成员。",
      "重聚米娅:伊森找到米娅，但她被感染并攻击伊森，伊森被迫自卫。",
      "逃离与追捕:伊森试图逃离贝克家，但被杰克·贝克抓住并囚禁。",
      "家庭晚餐:伊森在贝克家的晚餐桌上醒来，目睹贝克家族的疯狂行为，设法逃脱。",
      "地下室探索:伊森探索地下室，寻找逃生路径，遭遇变异生物。",
      "佐伊的帮助:伊森与贝克家的女儿佐伊合作，寻找解药以治愈米娅和自己。",
      "面对玛格丽特:伊森与玛格丽特·贝克对抗，成功击败她并继续寻找解药。",
      "卢卡斯的挑战:伊森进入卢卡斯·贝克设置的陷阱和谜题，成功逃脱并获得重要线索。",
      "最终对决:伊森面对伊芙琳，揭示她是感染的源头，成功消灭她并逃出贝克家。"
    ],
    "states": [
      [
 

In [ ]:
temp_save_file_path = '../data/temp_save.jsonl'
with jsonlines.open(temp_save_file_path, mode='a') as writer:
    writer.write(story_dict)

In [3]:
temp_save_file_path = '../data/temp_save.jsonl'

with jsonlines.open(temp_save_file_path, mode='r') as reader:
    story_dicts = [obj for obj in reader]
print(len(story_dicts))
story_dict = story_dicts[0]

1


In [4]:
# print(cleaned_plot_string)
print(json.dumps(story_dict,indent=2, ensure_ascii=False))  
#print(json.dumps(plot_dict,indent=2, ensure_ascii=False))  

{
  "Settings": {
    "story_id": "",
    "story_name": "生化危机7(Resident Evil 7)",
    "story_style": [
      "恐怖生存",
      "心理惊悚",
      "黑暗悬疑",
      "紧张刺激",
      "超自然元素",
      "家庭悲剧",
      "孤立无援"
    ],
    "leading_role": "伊森·温特斯",
    "story_desc": "故事发生在2017年，地点位于美国路易斯安那州的一个偏远农场，主角伊森·温特斯接到失踪三年的妻子米娅的神秘信息后，前往贝克家族的庄园寻找她。贝克家族被一种神秘的生物感染，变得极其危险和疯狂。伊森在探索过程中揭开了关于贝克家族和感染源的恐怖秘密。",
    "story_goal": "伊森的主要目标是找到并营救他的妻子米娅，同时揭开贝克家族背后隐藏的真相，并设法逃离这个充满危险的庄园。在这个过程中，他需要面对被感染的贝克家族成员和其他恐怖生物的威胁。",
    "story_chars": {},
    "storylines": [
      "收到消息:伊森·温特斯收到失踪三年的妻子米娅的录像，前往路易斯安那州的贝克家寻找她。",
      "贝克家探险:伊森进入贝克家，发现房子被感染，遭遇疯狂的贝克家族成员。",
      "重聚米娅:伊森找到米娅，但她被感染并攻击伊森，伊森被迫自卫。",
      "逃离与追捕:伊森试图逃离贝克家，但被杰克·贝克抓住并囚禁。",
      "家庭晚餐:伊森在贝克家的晚餐桌上醒来，目睹贝克家族的疯狂行为，设法逃脱。",
      "地下室探索:伊森探索地下室，寻找逃生路径，遭遇变异生物。",
      "佐伊的帮助:伊森与贝克家的女儿佐伊合作，寻找解药以治愈米娅和自己。",
      "面对玛格丽特:伊森与玛格丽特·贝克对抗，成功击败她并继续寻找解药。",
      "卢卡斯的挑战:伊森进入卢卡斯·贝克设置的陷阱和谜题，成功逃脱并获得重要线索。",
      "最终对决:伊森面对伊芙琳，揭示她是感染的源头，成功消灭她并逃出贝克家。"
    ],
    "states": [
      [
 

#### trigger refine

In [12]:
# 
# 读取信息
# for story_dict_i in story_dict_list:

Settings = story_dict["Settings"]
Episodes = story_dict["Episodes"]

story_name = Settings["story_name"]
story_style = Settings["story_style"]
leading_role = Settings["leading_role"]
story_desc = Settings["story_desc"]
story_goal = Settings["story_goal"]
story_chars = Settings["story_chars"]
states = Settings["states"]
story_language = Settings["story_language"]

# print(json.dumps(Settings,indent=2,ensure_ascii=False))
# print(json.dumps(Episodes[0],indent=2,ensure_ascii=False))

In [ ]:
# 评估plot的prompt
# 未使用 效果自身还可以
prompt_evaluate_episode=PromptTemplate(
    template = """请根据总体章节的信息严格分析当前章节的目标，场景和剧情摘要，并说明原因（尽可能严格，寻找问题与不足之处）
# 评价内容(可以参考但不限于以下内容)
1. 当前章节的目标，背景和剧情是否按照总目标前进
2. 是否总是以主角{leading_role}作为展开
3. 是否符合人物总体的行事风格以及观念
4. 剧情展开是否合理，有没有逻辑漏洞或者难以理解的发展过程
5. 剧情是否符合故事的总体风格

# 输出规范
 - 用{language}回答
 - 以dict的json格式重构
 - 格式如下: "main":{{"episode_goal":"**", "episode_scene":"**", "episode_desc":"**"}} '(**是对应的具体评价及原因)

# 上下文信息
 - 需要评估的原章节内容如下：
原章节的目标: {episode_goal}; 原章节的场景: {episode_scene}; 原章节的剧情摘要: {episode_desc},
 - 其他补充信息:
故事风格: {story_style}; 总体背景: {story_desc}; 主要目标: {story_goal}; 人物信息: {story_chars}
    """,
    input_variables=["leading_role","language","episode_goal","episode_scene","episode_desc","story_style","story_desc","story_goal","story_chars"],
)


# 评估triggers
# eva and refine 
# 未启用
prompt_evaluate_trigger=PromptTemplate(
    template = """你是一位资深故事结构分析师，专门负责分析改进章节间的逻辑衔接(跳转事件trigger)
# 核心目标
 - 根据总体章节信息严格评价当前章节跳转到下一情节的原因及跳转事件, 严格寻找问题与不足, 提供trigger的改写办法(保持当前格式，不修改章节内容), 精简回答
 - 验证是否符合「 当前plot → <IF>触发 → <THEN>衔接 → 下个plot 」时序链
 - 仅关注触发器<IF><THEN>结构, 忽略文学性, 保持分析客观性
    
# 重点关注项
 - 时间连贯性: <IF>是否存在于当前章结尾处延展的时间点? <THEN>是否构成独立于下章开篇的衔接事件? trigger是否起到承上启下的作用?
 - 剧情流程性: 两段plot通过当前跳转事件连接是否流畅自然? <IF>是否在当前情节之中可以达成，但是发生在当前情节之后的跳转条件?
 - 事件合理性: 跳转事件是否合理、符合逻辑，与故事背景/情节, 人物行为/风格一致?
 - 主体关联性: 触发条件是否由主角「{leading_role}」的行为直接/间接引发？
 - 逻辑完整性: <IF>是否为单一可验证条件, 且仅为条件而非事件描述? <THEN>是否包含独立事件而非章节内容复述?

# 修改规范
<IF>必须满足:
    - 是当前情节之后可能触发的原因, 不与当前plot内容重叠, 与当前plot流畅衔接
    - 只是一个触发的原因, 即仅包含触发条件, 而非具体事件
    - 包含主角行为影响
<THEN>必须满足:
    - 是下一章节发生之前的内容, 不与下一plot内容重叠, 与下一plot流畅衔接
    - 是<IF>达成后产生的事件结果, 只是一个触发原因导致的事件
整体上: 不要复述输入, 只需要介绍当前trigger的不足之处并说明原因, 提供改进办法
    「 当前plot → <IF>触发 → <THEN>衔接 → 下个plot 」这个顺序必须流畅连贯自然; 不要产生突兀转折, 重复剧情以及剧情断裂/脱节/中断
    如果有一个通用的语句: "当***, 则***", 那么<IF>就是「当***」的部分,<THEN>就是「则***」的部分
    
# 上下文背景
当前plot如下: {episode_desc}
下一plot摘要如下: {next_episode_desc}
需要修改的跳转事件如下: {trigger_condition}
故事风格: {story_style}; 世界观: {story_desc}; 主要人物: {story_chars}

# 输出格式
 - 严格使用JSON格式
 - 使用「{language}」回答
示例如下:
{{
    "不足":"(原trigger的不足)",
    "改进方法":"(提出你的改进方法)",
    "IF":"(改进后的if)",
    "THEN":"(改进后的THEN)"
}}
    """,
    input_variables=["leading_role","language","episode_desc","next_episode_desc","trigger_condition","story_style","story_desc","story_chars"],
)


# triggers eva only
# 
prompt_Eva_Score_Trigger=PromptTemplate(
    template = """你是一位资深故事结构分析师，专门负责负责章节间的逻辑衔接(跳转事件trigger)的评分, 根据总体章节信息严格评价当前章节跳转到下一情节的原因及跳转事件(验证是否符合「 当前plot → <IF>触发 → <THEN>衔接 → 下个plot 」时序链), 严格寻找问题与不足, 尽可能严厉, 给出专业评分(0-100整型数), 评分须严格遵循标准，扣分必须注明具体原因。
    
# 重点关注项
 - 时间连贯性(20分): <IF>是否存在于当前章结尾处延展的时间点? <THEN>是否构成独立于下章开篇的衔接事件? trigger是否起到承上启下的作用?
 - 剧情流程性(20分): 两段plot通过当前跳转事件连接是否流畅自然? <IF>是否在当前情节之中可以达成，但是发生在当前情节之后的跳转条件?
 - 事件合理性(20分): 跳转事件是否合理、符合逻辑，与故事背景/情节, 人物行为/风格一致?
 - 主体关联性(20分): 触发条件是否由主角「{leading_role}」的行为直接/间接引发？
 - 逻辑完整性(20分): <IF>是否为单一可验证条件, 且仅为条件而非事件描述? <THEN>是否包含独立事件而非章节内容复述?
 - 注意: 仅关注触发器<IF><THEN>结构, 忽略文学性, 保持分析客观性
 
# 上下文背景
当前plot如下: {episode_desc}
下一plot摘要如下: {next_episode_desc}
需要修改的跳转事件如下: {trigger_condition}
故事风格: {story_style}; 世界观: {story_desc}; 主要人物: {story_chars}

# 输出格式
 - 返回一个json格式的字典,包含两个键, 评分的原因以及最终的具体分数
示例如下:
{{
    "reason": "(各个指标的（评分/总分）以及扣分原因)"
    "score": 最终评分(仅数字)
}}
    """,
    input_variables=["leading_role","episode_desc","next_episode_desc","trigger_condition","story_style","story_desc","story_chars"],
)



# triggers opt only
# 
prompt_Eva_Opt_Trigger=PromptTemplate(
    template = """你是一位资深作家，专门负责分析改进章节间的逻辑衔接(跳转事件trigger), 接下来我将给你一个当前的trigger评价, 根据总体章节信息与评价，严格分析当前章节跳转到下一情节的原因及跳转事件中断问题与不足, 提供trigger的改写办法(保持当前格式，不修改章节内容), 精简回答

# 修改规范
仅关注触发器<IF><THEN>结构, 忽略文学性, 保持分析客观性
<IF>必须满足:
    - 包含主角行为影响(触发条件由主角「{leading_role}」的行为直接/间接引发)
    - 是<THEN>发生的直接原因, 是当前情节后可能产生的行为, 不与当前plot内容重叠, 与当前plot流畅衔接
    - <IF>存在于当前章结尾处延展的时间点, 即<IF>是在当前情节之中可以达成，但是发生在当前情节之后的跳转条件
    - <IF>为单一可验证条件, 且仅为条件而非事件描述(只是一个触发的原因, 即仅包含触发条件, 而非具体事件)
    - <IF>是<THEN>及以后事件发生的关键原因, 应该仅包含主角/或主角导致的一个关键动作
<THEN>必须满足:
    - 是<IF>产生的直接结果, 是下一章节发生之前的内容, 不与下一plot内容重叠, 与下一plot流畅衔接
    - <THEN>是构成独立于下章开篇的衔接事件
    - <THEN>包含独立事件而非章节内容复述
    - <THEN>是<IF>达成后产生的事件结果, 只是一个触发原因导致的事件(直接后继节点)
整体上: 
    - 不要复述输入, 只需要介绍当前trigger的不足之处并说明原因, 提供改进办法
    - 「 当前plot → <IF>触发 → <THEN>衔接 → 下个plot 」这个顺序必须流畅连贯自然; 不要产生突兀转折, 重复剧情以及剧情断裂/脱节/中断
    - 如果有一个通用的语句: "当***, 则***", 那么<IF>就是「当***」的部分,<THEN>就是「则***」的部分
    - trigger需要起到承上启下的作用, 两段plot通过当前跳转事件连接的流畅自然
    - 修改后的跳转事件合理、符合逻辑，与故事背景/情节, 人物行为/风格一致
    - <IF>与<THEN>不要与同章节的其他跳转条件重复, 触发原因和产生的结果都应该具有较大差异
    
# 上下文背景
当前plot如下: {episode_desc}
下一plot摘要如下: {next_episode_desc}
需要修改的跳转事件如下: {trigger_condition}
跳转事件评估结果如下: {trigger_eva_think}
{other_triggers}
故事风格: {story_style}; 世界观: {story_desc}; 主要人物: {story_chars}

# 输出格式
 - 严格使用JSON格式,仅输出一个dict, 不要输出其他内容
 - 使用「{language}」回答
示例如下:
```
{{
    "改进方法":"(提出你的改进方法与依据)",
    "IF":"(改进后的IF)",
    "THEN":"(改进后的THEN)"
}}
```
    """,
    input_variables=["leading_role","language","episode_desc","next_episode_desc","trigger_condition","story_style","story_desc","story_chars","trigger_eva_think","other_triggers"],
)



In [ ]:
story_dict_new = copy.deepcopy(story_dict)
trigger_opt_threshold = 80

for j in tqdm(range(len(Episodes))):
    episode = Episodes[j]

    # 判断总体章节信息
    # 不太需要, 效果本身还可以
    if False:
        chain_evaluate_episode = prompt_evaluate_episode | llm_agent
        res_evaluate_episode = chain_evaluate_episode.invoke({"leading_role":leading_role,"language":story_language,"episode_goal":episode["episode_goal"],"episode_scene":episode["episode_scene"],"episode_desc":episode["episode_desc"],"story_style":story_style,"story_desc":story_desc,"story_goal":story_goal,"story_chars":story_chars}).content
        try:
            cleaned_json_str0 = res_evaluate_episode.split('```json')[1].split('```')[0].strip()
        except:
            cleaned_json_str0 = res_evaluate_episode
        #print(cleaned_json_str0)
        res0_dict = {}
        res0_dict = json.loads(cleaned_json_str0)["main"]
            
        # print(json.dumps(res0_dict,indent=2,ensure_ascii=False))
        story_dict_new["Episodes"][j]["evaluate"] = res0_dict


    # 判断triggers
    
    triggers = episode["triggers"]
    if (not triggers == []) and (not triggers == [{}]):
        # continue
        # print(triggers)
        other_triggers = ""
        for k in range(len(triggers)):
            # break
            trigger = triggers[k]
            # 这里看一下要不要-1
            next_episode = Episodes[trigger["next_episode"]-1]
            
            # chain_evaluate_trigger = prompt_evaluate_trigger | llm_agent
            # res_evaluate_trigger = chain_evaluate_trigger.invoke({"leading_role":leading_role,"language=story_language,"episode_desc=episode["episode_desc"],next_episode_desc=next_episode["episode_desc"],"trigger_condition":trigger["condition"],"story_style":story_style,"story_desc":story_desc,"story_goal":story_goal,"story_chars":story_chars}).content

            prompt6_1 = prompt_Eva_Score_Trigger.format(leading_role=leading_role,episode_desc=episode["episode_desc"],next_episode_desc=next_episode["episode_desc"],trigger_condition=trigger["condition"],story_style=story_style,story_desc=story_desc,story_chars=story_chars)

            res_Eva_think_1, res_Eva_Score_Tri1 = llm_chat_analysis("gpt-4o", prompt6_1, "dict")
            story_dict_new["Episodes"][j]["triggers"][k]["eva_reason_ori"] = res_Eva_Score_Tri1["reason"]
            res_Eva_Score_Tri1["score"] = int(res_Eva_Score_Tri1["score"])
            story_dict_new["Episodes"][j]["triggers"][k]["eva_score_ori"] = res_Eva_Score_Tri1["score"]

            # print(res_Eva_think_1)
            # print(json.dumps(res_Eva_Score_Tri1,indent=2,ensure_ascii=False))

            if res_Eva_Score_Tri1["score"]<trigger_opt_threshold:
                # prompt6 = prompt_evaluate_trigger.format(leading_role=leading_role,language=story_language,episode_desc=episode["episode_desc"],next_episode_desc=next_episode["episode_desc"],trigger_condition=trigger["condition"],story_style=story_style,story_desc=story_desc,story_chars=story_chars)
                # res_think, res_evaluate_trigger = llm_chat_analysis("deepseek-r1", prompt6, "dict")
                # res_evaluate_trigger["eva_think"] = res_think

                prompt6_2 = prompt_Eva_Opt_Trigger.format(leading_role=leading_role,language=story_language,episode_desc=episode["episode_desc"],next_episode_desc=next_episode["episode_desc"],trigger_condition=trigger["condition"],story_style=story_style,story_desc=story_desc,story_chars=story_chars,trigger_eva_think=res_Eva_Score_Tri1["reason"],other_triggers=other_triggers)
                res_Eva_think_2, res_Eva_Opt_Tri1 = llm_chat_analysis("deepseek-r1", prompt6_2, "dict")
                

                # print("当前情节: ", episode["episode_desc"])
                # print("下一情节: ", next_episode["episode_desc"])
                # print("trigger: ", trigger["condition"])
                # print(res_Eva_think_2)
                # print(json.dumps(res_Eva_Opt_Tri1,indent=2,ensure_ascii=False))

                # break
                story_dict_new["Episodes"][j]["triggers"][k]["eva_opt_ori"] = res_Eva_Opt_Tri1["改进方法"]
                story_dict_new["Episodes"][j]["triggers"][k]["old_con"] = trigger["condition"]
                new_trigger_con = "<IF>" + res_Eva_Opt_Tri1["IF"] + " <THEN>" + res_Eva_Opt_Tri1["THEN"]
                story_dict_new["Episodes"][j]["triggers"][k]["condition"] = new_trigger_con
                # print(json.dumps( story_dict_new["Episodes"][j]["triggers"][k],indent=2,ensure_ascii=False))

                prompt6_3 = prompt_Eva_Score_Trigger.format(leading_role=leading_role,episode_desc=episode["episode_desc"],next_episode_desc=next_episode["episode_desc"],trigger_condition=new_trigger_con,story_style=story_style,story_desc=story_desc,story_chars=story_chars)
                res_Eva_think_3, res_Eva_Score_Tri2 = llm_chat_analysis("gpt-4o", prompt6_3, "dict")
                story_dict_new["Episodes"][j]["triggers"][k]["eva_reason_opt"] = res_Eva_Score_Tri2["reason"]
                res_Eva_Score_Tri2["score"] = int(res_Eva_Score_Tri2["score"])
                story_dict_new["Episodes"][j]["triggers"][k]["eva_score_opt"] = res_Eva_Score_Tri2["score"]

                # print(res_Eva_think_3)
                # print(json.dumps(res_Eva_Score_Tri2,indent=2,ensure_ascii=False))

                # break

            if not other_triggers:
                other_triggers = "当前章节其他的的triggers如下:\n"
            other_triggers+=story_dict_new["Episodes"][j]["triggers"][k]["condition"]+"\n"
            # break
    # 结局生成
    else:
        # print(json.dumps(episode,indent=2,ensure_ascii=False))
        
        prompt6_4 = prompt_EndTri_write.format(novel_name= story_name,leading_role=leading_role,episode_desc=episode["episode_desc"],story_style=story_style,story_desc=story_desc,story_chars=story_chars,language = "中文")
        print(len(prompt6_4))
        res_Gen_think_4, res_Gen_End= llm_chat_analysis("gpt-4o", prompt6_4, "dict")
        print("episode_desc: ",episode["episode_desc"])
        print("res_Gen_think_4: ",res_Gen_think_4)
        print(json.dumps(res_Gen_End,indent=2,ensure_ascii=False))
        # story_dict_new["Episodes"][j]["episode_desc"] = res_Gen_End["最终章流程"] 
        story_dict_new["Episodes"][j]["triggers"]["condition"] = "<IF>" +res_Gen_End["结局触发点"] + " <THEN>" + res_Gen_End["结局演出"] 
        story_dict_new["Episodes"][j]["triggers"]["next_episode"] = -1


        break

    # break
# print(Settings)

story_dict = story_dict_new
#story_dict_list_new[i] = story_dict_new


# deepseek 评价单情节评估评价时间30s

  0%|          | 0/19 [00:00<?, ?it/s]


NameError: name 'prompt_Eva_Score_Trigger' is not defined

In [39]:
print(llm_chat_analysis("gpt-4o", "1", "str"))
print(llm_chat_analysis("deepseek-r1", "1", "str"))

('', 'Hello! How can I assist you today?')


2025-03-08 16:06:44.014 | ERROR    | llm:send:431 - [DeepSeek] API request error: status_code=400, {"error":{"message":"Invalid max_tokens value, the valid range of max_tokens is [1, 8192]","type":"invalid_request_error","param":null,"code":"invalid_request_error"}}, retrying...
2025-03-08 16:06:45.244 | ERROR    | llm:send:431 - [DeepSeek] API request error: status_code=400, {"error":{"message":"Invalid max_tokens value, the valid range of max_tokens is [1, 8192]","type":"invalid_request_error","param":null,"code":"invalid_request_error"}}, retrying...
2025-03-08 16:06:46.498 | ERROR    | llm:send:431 - [DeepSeek] API request error: status_code=400, {"error":{"message":"Invalid max_tokens value, the valid range of max_tokens is [1, 8192]","type":"invalid_request_error","param":null,"code":"invalid_request_error"}}, retrying...


TimeoutError: Max retry exceeded

In [29]:
prompt_EndTri_write = PromptTemplate(
    template = """你是一个作家, 我将会给你一个结局剧情, 基于「{novel_name}」世界观，用{language}分析

# 核心要求
 - 你需要生成三个部分「最终章流程」「结局触发点」「结局演出」
 - 完全基于当前结局剧情,不能改写，只能扩写
 - 「最终章流程」是可互动剧情
 - 「结局触发点」只是一个条件或者触发的行为等,不包括后续内容, 尽可能短(例如:击败boss(注意只是击败这个条件而非开始对战), 主角或者某个npc阵亡, 找到某个关键道具, 对战持续多久后等情况, 需要和当前剧情摘要保持一致))
 - 「结局演出」应是最终一个结局，单一事件(「结局触发点」产生的结果),主角不能做出影响事件的抉择, 这部分可以留有悬念
 - 如果原结局剧情包含了结尾，需要将原结局剧情拆成这三个部分; 如果没有包含结尾, 将原结局剧情整理成「最终章流程」,在流程之后续写「结局触发点」「结局演出」两个部分
 - 简洁明了地阐述从当前故事结束的关键原因
 - 人物动机需符合性格设定, 事件发展需符合世界观规则
 - 避免提及「玩家」「游戏」「剧情中」「故事」元叙事词汇
   
# 格式示例
json格式的dict
```
{{
    "最终章流程":"(保留所有可操作内容（战斗/解谜/对话分支）)",
    "结局触发点":"(达成某项事件或者触发某个条件",
    "结局演出":"(达到结局触发点后触发的最终结局，不能互动,需要与主角强相关「{leading_role}」)",
}}
```

# 上下文内容
 - 当前plot如下: {episode_desc}
 - 故事风格: {story_style}; 世界观: {story_desc}; 主要人物: {story_chars}
    """,
    input_variables=["novel_name","leading_role","episode_desc", "story_style","story_desc","story_chars","language"],
)


In [69]:
print(res_Eva_Opt_Tri1)
print(type(res_Eva_Opt_Tri1))

{
    "改进方法":"原trigger存在线索发现未前置铺垫、条件复杂度过高、衔接事件独立性不足的问题。改进方案将触发条件锚定在主角当前章节结尾已完成的动作(推门)，通过环境反馈自然衔接至下一章节的危机场景，既符合时间连贯性又增强紧张感。",
    "IF":"伊森·温特斯推开贝克家生锈铁门时触发了隐蔽的警报装置",
    "THEN":"刺耳的警报声打破死寂，阁楼传来重物坠地的闷响，迫使伊森立即躲进最近的储物间，透过门缝看见手持铁锤的扭曲人影正从楼梯拐角逼近"
}

改进依据：
1. IF条件调整：将触发点从「找到线索」改为「推门触发警报」，既符合当前章节结尾动作(推门)，又自然产生时间延展性
2. 增强因果逻辑：物理性触发装置(15.7%恐怖游戏常用手法)直接导致敌人出现，比「找到线索」更具动作关联性
3. THEN重构：通过「警报-躲藏-目击」三阶段事件链，既保持独立衔接事件属性(占叙事时长约2-3分钟)，又与下章开篇的「探索躲避」形成无缝衔接
4. 符合世界观设定：被感染家族设置原始警报系统(挂铃/铁丝)的细节，与后续揭示的「贝克家刻意引诱受害者」的暗线形成呼应
<class 'str'>


#### 总体人物生成

In [28]:
#print(prompt_role_list)

In [23]:
# 总体人物介绍


prompt_role_list=PromptTemplate(
    template = """从给定列表中提取游戏「{novel_name}」中的核心人物，按剧情权重降序输出，满足：
# 处理流程
    1. 名称预处理(名称标准化): 将人物的名称统一为最通用完整名称(可能包括不同译名,别名,昵称,英文名等)
    2. 重要性筛选: 第一个人物应该是故事中的主角「{leading_role}」,其他人物按重要性/剧情出现频率排序, 最终保留不超过7个角色
    3. 信息结构化: 按身份(基础身份+隐藏身份(若存在)), 性格(显性特质+隐性矛盾), 经历(人物背景,出场状态,关键转折点,持续状态等方面)三大方面依次介绍

# 输出规范
    - 使用json结构, list格式, '["xx:xx", "xx:xx", ...]'重构
    - 使用"角色名:属性组"格式，属性组包含:身份标签,性格,经历
    - 条目数≤7, 按综合权重值排序
    - 不要提到"玩家","在xx游戏中", "在剧情中"等令人出戏的字眼, 不要提到本游戏具体名称
    - 用「{language}」回答

# 输出示例
```
[
    "劳拉·克劳馥": "主角，21岁，刚踏出校园。充满自由精神和荒野情怀，坚韧不拔，作风实际，头脑冷静。在寻找邪马台古国的途中，劳拉所在的船只遭遇风暴失事，她漂流到一个未知的岛屿上，孤身一人在岛上醒来，开始她的求生之旅,在岛上，劳拉学会了生存，经历了从一名经验不足的年轻女孩到一个能够独当一面的冒险家的成长过程。在这个过程中，劳拉不仅要面对凶猛的野生动物和敌对的势力，还要揭开岛上的秘密，寻找失踪的同伴，并最终战胜邪教组织。",
    "珊曼莎·西村": "实习纪录片制片人，也是与劳拉从寄宿学校一直读到伦敦大学的同窗好友。为人大大咧咧，习惯过着奢侈阔绰的生活。拥有葡萄牙和日本的双重国籍，家境殷实。由于她是太阳女王卑弥呼的后裔，所以马蒂亚斯企图将卑弥呼的灵魂植入她体内以复活太阳女王。",
    "詹姆斯·惠特曼": "考古学家，也是当红考古电视节目“惠特曼的世界”主持人。久负盛名但是自私冷漠，对财富和名声有着异呼寻常的渴望，心胸狭窄，为了私利不择手段。坚忍号的这次探险计划其实就是为“惠特曼的世界”下一季做准备。劳拉原来十分崇拜惠特曼博士，但在荒岛遇险过程中惠特曼所表现出的自私冷漠以及背叛使劳拉看清了他的真面目。",
    "马蒂亚斯": "反派boss，太阳兄弟会的首领。神经怪异，但行事却条理清晰，冷酷无情，为了实现自己的目标不惜牺牲他人。1982年，马蒂亚斯驾机迫降在小岛附近海域。在逃亡过程中，他发现了岛上隐藏的邪马台古国遗址，并得知太阳女王拥有不可思议的魔力。31年的孤岛生活使他变得疯狂而且偏执，他采用威胁和恐吓手段笼统了一帮遇难者组成太阳兄弟会，还在女王遗址外修建了一座破烂小城。当他知道坚忍号失事船员里的珊曼莎是卑弥呼后裔后，就开始了丧心病狂的活人献祭阴谋。",
]
```
需要整理的人物列表如下:{role_list}
    """,
    input_variables=["novel_name","role_list","leading_role","language"],
)

In [30]:
#print(role_list_str)
#print(role_list_str[1])

In [ ]:
# 最后整体补齐story所有人物信息
# 单线多线都有

#print(json.dumps(story_dict,indent=2, ensure_ascii=False))  
# 统计一下总人物
role_str = ""  
print(f"if_role_known: {if_role_known}")

# 第一种是根据人物列表直接总结，认为llm知道所有人物信息
if if_role_known:
    role_str_set = set()  
    for plot in story_dict["Episodes"]:  
        for role in plot["episode_chars"]:  
            if role not in role_str_set: 
                role_str += f"{role}, " 
                role_str_set.add(role)  #
    role_str.strip().strip(",")
    print("role_list: ",role_str)

# 第二种更适用于小说llm, llm不了解人物时根据所有人物状态总结
else:
    for i in range(len(story_dict["Episodes"])):  
        plot = story_dict["Episodes"][i]
        role_str += "故事的第{i}幕出场的人物以及状态如下:\n"
        for role in plot["episode_chars"]:  
            role_str += role + ": " + plot["episode_chars"][role] + "\n"

    role_str.strip().strip("\n")
    print("role_list: ",role_str)

# chain_role_list = prompt_role_list | llm_agent
# res_role_list = chain_role_list.invoke({"novel_name": novel_name, "role_list": role_str, "language":language}).content
    
prompt7 = prompt_role_list.format(novel_name=novel_name, role_list=role_str,leading_role=leading_role,language=story_language)
res_think, res_story_all_role = llm_chat_analysis("deepseek-r1", prompt7, "dict")

story_dict["Settings"]["story_chars"] = res_story_all_role


print(json.dumps(story_dict["Settings"]["story_chars"],indent=2, ensure_ascii=False))  
#print(json.dumps(story_dict,indent=2, ensure_ascii=False))  

# deepseek 用时 60-100s

if_role_known: True
role_list:  伊森·温特斯: 坚韧且执着，充满对妻子的爱和对真相的渴望。此时他被妻子米娅的录像吸引，决心前往贝克家寻找她，尽管对即将面对的危险一无所知。, 米娅·温特斯: 神秘且复杂，录像中显得焦虑和绝望。她失踪三年后突然通过录像联系伊森，似乎被困在贝克家中，处于一种未知的危险境地。, 伊森·温特斯: 坚韧不拔，充满求生欲望的普通人。他为了寻找失踪的妻子米娅而来到贝克家，面对超自然的恐怖和疯狂的敌人，他必须保持冷静并找到真相。, 杰克·贝克: 贝克家族的父亲，性格暴躁且不可预测。受感染的影响，他变得极其危险，常常在房子中追捕入侵者，试图将他们赶出或消灭。, 玛格丽特·贝克: 贝克家族的母亲，表现出极端的偏执和敌意。她对房子的感染有着特殊的联系，常常在阴暗的角落中徘徊，守护着她的家园。, 卢卡斯·贝克: 贝克家族的儿子，狡猾且具有恶作剧心理。他喜欢设置陷阱和玩弄入侵者，享受他们的恐惧和困惑。, 伊森·温特斯: 坚韧且富有决心的普通人，渴望找到失踪的妻子米娅。在面对庄园中的恐怖和危险时，他表现出极大的勇气和毅力。, 米娅·温特斯: 伊森的妻子，性格复杂，受感染后变得不稳定。她在失踪多年后被伊森找到，但此时的她已经被感染，处于一种危险和不可预测的状态。, 伊森·温特斯: 坚韧不拔，勇敢且机智。他是一个普通人，但为了寻找失踪的妻子米娅而来到贝克家。在被困于此时，他表现出强烈的求生欲望和对妻子的深切关心。, 杰克·贝克: 残忍，疯狂且极具威胁性。贝克家的家长，受到某种神秘力量的影响，变得极其暴力和不可预测。他对伊森展开了无情的追捕，试图阻止他逃离。, 伊森·温特斯: 坚韧而富有求生意志的普通人。他为了寻找失踪的妻子米娅而来到贝克家，面对家族的疯狂行为，他必须保持冷静，寻找逃脱的机会。, 杰克·贝克: 贝克家族的父亲，性格暴躁且具有攻击性。他似乎已经失去了理智，对伊森充满敌意，表现出极端的暴力倾向。, 玛格丽特·贝克: 贝克家族的母亲，性格阴郁且神秘。她对家族的异常行为视而不见，似乎在暗中策划着什么。, 伊森·温特斯: 坚韧不拔，勇敢且富有求生本能。此时的伊森正处于极度紧张和警觉的状态，因为他在寻找逃生路径的过程中不断遭遇危险。他的目标是尽快逃离这个充满恐怖的地方，找到安全的出路。, 伊森·温特斯: 坚韧不拔，勇敢且富有求生欲望

In [98]:
#print(res_role_list)
date_md = str(datetime.now().strftime("%m-%d")).replace("-","_")
print(f"data_now: {date_md}")
game_single_file_path = '../data/game_gen_11_23.jsonl'
game_single_file_path = f'../data/game_gen_{date_md}.jsonl'
print(f"save file path: {game_single_file_path}")
with jsonlines.open(game_single_file_path, mode='a') as writer:
    writer.write(story_dict)

data_now: 02_27
save file path: ../data/game_gen_02_27.jsonl


In [ ]:

# 单线/多线（改一下前面的参数）
# 这里按执行


当前日期: 12_14


In [99]:
print(json.dumps(story_dict,indent=2,ensure_ascii=False))

{
  "Settings": {
    "story_id": "",
    "story_name": "生化危机7(Resident Evil 7)",
    "story_style": [
      "恐怖生存",
      "心理惊悚",
      "黑暗悬疑",
      "紧张刺激",
      "超自然元素",
      "家庭悲剧",
      "孤立无援"
    ],
    "leading_role": "伊森·温特斯",
    "story_desc": "故事发生在2017年，地点位于美国路易斯安那州的一个偏远农场，主角伊森·温特斯接到失踪三年的妻子米娅的神秘信息后，前往贝克家族的庄园寻找她。贝克家族被一种神秘的生物感染，变得极其危险和疯狂。伊森在探索过程中揭开了关于贝克家族和感染源的恐怖秘密。",
    "story_goal": "伊森的主要目标是找到并营救他的妻子米娅，同时揭开贝克家族背后隐藏的真相，并设法逃离这个充满危险的庄园。在这个过程中，他需要面对被感染的贝克家族成员和其他恐怖生物的威胁。",
    "story_chars": {
      "伊森·温特斯": "普通工程师，生化感染事件的幸存者。坚韧不拔、富有责任感，兼具普通人的脆弱性与保护家人的决绝。因妻子米娅失踪前往路易斯安那州农场，发现贝克家族异变后被迫卷入求生斗争。经历断肢再生、血清抉择、对抗伊芙琳等事件，最终以自我牺牲摧毁菌核。",
      "米娅·温特斯": "前秘密特工/伊森妻子，E型霉菌感染者。矛盾纠结中保有良知，清醒时坚毅果敢。三年前因护送伊芙琳任务失败被囚禁，感染后时而被控制攻击伊森，时而在血清作用下恢复理智。关键转折在于选择牺牲自己换取伊森逃生。",
      "杰克·贝克": "农场主/霉菌感染者，家族支配者。狂暴表象下残留人性，具备扭曲幽默感。原为救伊芙琳反被感染，身体持续变异仍保持再生能力。多次追杀伊森未果，最终在车库决战中被彻底消灭。",
      "玛格丽特·贝克": "家庭主妇/霉菌共生体。偏执阴郁与母性本能交织，通过虫群操控守护宅邸。长期与霉菌深度融合导致身体昆虫化，最终在温室被火焰喷射器终结。",
      "卢卡斯·贝克": "家族次子/秘密实验协助者。病态天才兼具施虐倾向，沉迷设计致命机关。

### 多线划分(不需要了)


In [ ]:
# 对于string重组织，以便重新生成多线剧情，但是单线不需要，直接扩张

if not combine_plot_num == 0:
    com_plot_list = [main_plot_list["main"][i:i + combine_plot_num] for i in range(0, len(main_plot_list["main"]), combine_plot_num)]
else:
    com_plot_list = [main_plot_list["main"]]

if True:
    for i in range(len(com_plot_list)):
        sub_plot = com_plot_list[i]
        plot_str0 = ""
        plot_str1 = ""
        for item in sub_plot:
            item = item.split(":")
            plot_str0 += item[0].strip() + ", "
            plot_str1 += item[1].strip() + "\n"
        com_plot_list[i] = plot_str0.strip(", ") + "\n" + plot_str1
else:  
    com_plot_list = ['\n'.join(sub_plot) for sub_plot in com_plot_list]

print(com_plot_list)

In [ ]:

    
print(json.dumps(com_plot_list,indent=2, ensure_ascii=False))

# 存储
single_plot_dict = {"name":novel_name,**novel_background,"plot":[],"story_type":"game","language":language,"parameter":parameter_set}

# 这里有一个二层循环，对总章节的所有内容分开生成内容

for i in range(len(com_plot_list)):

    com_plot = com_plot_list[i]
    #print(f"重划分子剧情 - {i}:\n{com_plot}")
    chain2 = prompt_combine_plot | llm_agent
    res2 = chain2.invoke({"novel_name": novel_name, "plot_combine": com_plot,"seg_main_plot_num":seg_main_plot_num, "language":language}).content
        
    try:
        cleaned_json_string2 = res2.split('```json')[1].split('```')[0].strip()
        combine_sub_plot = json.loads(cleaned_json_string2)["main"]
    except:
        cleaned_json_string3 = ""  
              
    #print("段落子剧情: ",json.dumps(combine_sub_plot,indent=2, ensure_ascii=False))
    single_sub_plot = {"main":com_plot,"sub_single":combine_sub_plot}
    print("single_plot_new: ",json.dumps(single_sub_plot,indent=2, ensure_ascii=False))
    single_plot_dict["plot"].append(single_sub_plot)
    print("===========================")
    #break

with open(f'{data_folder}/game_sub_plot_see.jsonl', 'a') as f:
    json.dump(single_plot_dict, f, ensure_ascii=False, indent=4)
    # 在每个 JSON 对象后面添加换行符
    f.write('\n')
with open(f'{data_folder}/game_sub_plot.jsonl', 'a') as f:
    json.dump(single_plot_dict, f, ensure_ascii=False)
    # 在每个 JSON 对象后面添加换行符
    f.write('\n')
    



In [ ]:
# 中间存储，防止中断丢失数据
 
single_plot_dict1 = []
with jsonlines.open(f'{data_folder}/game_sub_plot.jsonl', 'r') as reader:
    # 逐行读取文件
    for obj in reader:
        # 将每个JSON对象存储为字典
        single_plot_dict1.append(obj)
single_plot_dict = single_plot_dict1[-1]
#print(json.dumps(single_plot_dict, indent=2, ensure_ascii=False))

In [ ]:
prompt_gen_multi_plot=PromptTemplate(
    template = """
        在{novel_name}游戏中，你将将现有的单线剧情（该部分的原先的主要剧情为{main_plot}）重写成一个多分支剧情树。以下是你的任务：

        注意：重点在重写而非续写，需要在原有剧情之间添加新剧情，改变原有剧情序号
        不要重新将之前的plot重新先写一边再拓展

        1.剧情重写：重写当前剧情，添加新剧情，新结局，并使所有剧情连贯，使其包含多个分支（这里的分支不只是结局，包括中间也可能产生分支）。
        2.详细描述：详细写下每个情节，包括发生的事件、跳转原因、触发条件和结局。需要考虑不同选择导致剧情走向的改变。
        3.角色一致：确保新情节与角色特点相符。如果两个分支指向同一剧情，它们的跳转逻辑必须各自合理。
        4.剧情连贯：确保从起始事件plot1可以达到其他所有事件，而且所有事件都可以通过不同的决策路径到达最终几个结局。
        5.跳转合理：如果两个分支指向同一个剧情，那么这两个分支的跳转必须符合不同分支的剧情逻辑。所有结局要和对应剧情逻辑保持一致。
        6.结局多样：创造显著不同的多个结局，结局不能同质化，确保不同结局的人物命运和事件结果有显著差异。确保不同的重要事件能触发不同的事件链，导致多样的结局。

        格式要求如下：
        不同结尾或者某一事件产生的分支剧情的数量应该至少{branch_num_min}个，至多{branch_num_max}个
        每个剧情前用(plot *)，在剧情后用(from plot **)指示从何剧情跳转而来,*和**是对应剧情序号
        你的输出应该从plot1开始，序号需要连贯
        
        下面是一个例子
        ```
        (plot 1)寻找希里:杰洛特寻找他的养女希里，她被狂猎追捕。(begin)
        (plot 2)战场的鲜血:希里情绪低落，需要做出选择以将决定希里是否能够控制自己的力量。(from plot 1)
        (plot 3)希里女王支线:"战场上的鲜血"任务选择去带希里参见皇帝。(from plot 2)
        (plot 4)希里猎魔人支线:"战场上的鲜血"任务选择去追杀狂猎。(from plot 2)
        (plot 5)铸剑:杰洛特为希里铸造一把钢剑，并在百果园的酒馆将剑送给她，象征着她成为一名真正的猎魔人。(from plot 4)
        (plot 6)游历:希里会独立去探索世界，而杰洛特则会继续他的猎魔人生涯。(from plot 5)
        (plot 7)共同作战:希里与杰洛特一起成为猎魔人，为追杀狂猎做准备(from plot 5)
        (plot 8)聆听故事:杰洛特听说陶森特有一位猎魔大师结束了混乱的吸血鬼屠杀事件，决定去找他交流经验(from plot 5)
        (plot 9)...
        (end 1)下落不明:希里在面对白霜时下落不明。杰洛特在独自寻找希里的项链时被食尸鬼群包围淹没(from plot 2)
        (end 2)希里的堕落:希里在成为女王后，被权力所诱惑，与狂猎达成交易，最终同流合污(from plot 3)
        (end 3)胜利与牺牲:虽然最终击败了狂猎，但是希里为了保护她所爱的人选择了牺牲自己(from plot 7)
        (end 4)重见故友:杰洛特发现猎魔大师就是希里。她在庄园的树下与杰洛特重逢(from plot 8)
        (end 5)...
        ```
        游戏发生的背景如下:{game_background}
        游戏的目标如下:{game_target}
        用{language}回答,原单线剧情树如下(重写):{sub_plot}
    """,
    input_variables=["novel_name","main_plot","sub_plot","branch_num_min","branch_num_max","game_background","game_target","language"],
 )  


# 在产生分支剧情时原剧情应该有所提示，分支剧情的输出位置尽可能贴近分支节点
# 比如在之前的剧情中间插入分支，而不是只在末尾加入分支，并将目前整体的剧情重构

In [ ]:
# 一个是跳转多结局的问题，是做出一个事件就会导致后面截然不同
# 还是需要达成某几个先决条件才会影响结局改变

# 另一个就是 如果子剧情产生不同的结局会导致不同块结局的割裂

# 现在先搭好多线剧情树，然后之后为剧情树生成事件背景以及人物对话
reset_text ="清除上下文"
#response = llm_agent.invoke(reset_text)
#print(response)

In [ ]:
single_plot_dict
chain3 = prompt_gen_multi_plot | llm_agent

plot_background = single_plot_dict["background"]
plot_target = single_plot_dict["target"]
print(f"background: {plot_background}")
print(f"game target: {plot_target}")

#print(json.dumps(single_plot_dict, indent=2, ensure_ascii=False))


for i in tqdm(range(len(single_plot_dict["plot"]))):


    plot_main = single_plot_dict["plot"][i]["main"]
    plot_sub = '\n'.join(single_plot_dict["plot"][i]["sub_single"])


    #print("主线: ",plot_main)
    #print("主线: "plot_sub)
    
    #break
    #["novel_name","main_plot","sub_plot","branch_num_min","branch_num_max","game_background","game_target","language"],


    
    res3 = chain3.invoke({"novel_name": novel_name, "main_plot":plot_main,"sub_plot": plot_sub,"branch_num_min":branch_num-1, "branch_num_max":branch_num+1,"game_background":plot_background,"game_target":plot_target,"language":language})
    #print(res3.response_metadata["token_usage"])

    
    gen_multi_plot = res3.content.split('```\n')[1].split('```')[0].strip()
    #gen_multi_plot_cut = gen_multi_plot.split("\n")
    
    #print(gen_multi_plot_cut)
    gen_multi_plot_cut = gen_multi_plot.split("\n")
    # 处理一下文本

    process_txet_flag = True
    if process_txet_flag:
    # 这里要不要处理缩进
    # 缩进是好的，但是生成不一定有
        for j in range(len(gen_multi_plot_cut)):
            item = gen_multi_plot_cut[j]
            item = item.strip().strip("-").strip()
            #print(item)

            # 这里要不要空行
            gen_multi_plot_cut[j] = item



    single_plot_dict["episode"][i]["gen_multi"] = gen_multi_plot_cut



    #break
    #plot_tree_multi_path =  f"{data_folder}/{novel_name}_gen_multi.txt"
    #with open(plot_tree_multi_path, 'w', encoding='utf-8') as f:
    #    f.write( plot_tree_multi_path )

    #break
    #"""
with open(f'{data_folder}/game_gen_plot.jsonl', 'a') as f:
    # 使用jsonlines.Writer写入新的一行
    writer = jsonlines.Writer(f)
    writer.write(single_plot_dict)

In [ ]:
# 原文本会不变

gen_multi_plot= res3.content
gen_multi_plot= gen_multi_plot.split('```\n')[1].split('```')[0].strip()
print(gen_multi_plot)
single_plot_dict["plot"][i]["gen_multi"] = gen_multi_plot.split("\n")
with open(f'{data_folder}/{novel_name.replace(" ","_")}_gen_plot.jsonl', 'a') as f:
    # 使用jsonlines.Writer写入新的一行
    writer = jsonlines.Writer(f)
    writer.write(single_plot_dict)